
```                                                                        
(declare-datatypes ((C 0)) (((C (re Real) (im Real)))))
(declare-fun sin (Real) Real)
(declare-fun cos (Real) Real)
(declare-fun mul (C C) C)
(declare-fun expi (Real) C)
(assert (forall ((t Real) (s Real))
  (let ((a!1 (C (- (* (cos t) (cos s)) (* (sin t) (sin s)))
                (+ (* (cos t) (sin s)) (* (sin t) (cos s))))))
    (= a!1 (C (cos (+ t s)) (sin (+ t s)))))))
(assert (forall ((t Real) (s Real))
  (let ((a!1 (C (- (* (cos t) (cos s)) (* (sin t) (sin s)))
                (+ (* (cos t) (sin s)) (* (sin t) (cos s))))))
    (= (mul (expi t) (expi s)) a!1))))
(assert (let ((a!1 (forall ((t Real) (s Real))
             (= (mul (expi t) (expi s)) (C (cos (+ t s)) (sin (+ t s)))))))
  (not a!1)))
(check-sat)

```
z3 can't do this one. vampire can. Porbably an ematching through + * problem

# Staged Metaprogramming
I had a post somewehre on this point. That programming z3 in python is staged metaprogramming.
The meta system python can do partial evaluation.


Can I eval z3 expressions in python metalayer? the other options is quoting 
Getting more homicornicity




In [1]:
from z3 import *

In [ ]:
def mypow(x:int,n:int) -> int:
    assert n >= 0
    if n == 0:
        return 1
    else:
        x * mypow(x,n-1)

# accumulator version?

# string version
# strings are a universal but somewhat structure free rep of code.
Code = str
def mypow2(n:int, x:Code) -> Code:
    if x == 0:
        return "1"
    else:
        f"{x} * {mypow2(x,n-1)}"

mypow = lambda x,n: 1 if n <= 0 else x * mypow(x,n-1)

mypow = Function("mypow", IntSort(), IntSort(), IntSort())
mypow_def = ForAll([x,n], mypow(n, x) == If(n <= 0, 1, x * mypow(n-1, x)))

# Partially evaled
def mypow(x:ExprRef, n:int) -> ExprRef:
    if n == 0:
        return IntVal(1)
    else:
        return x * mypow(x,n-1)




They could be named in previous stages maybe?

Am I pushing the choose operation to meta level instead of internalizing?

The idea was a genheralization of typing problems. I want typing discharged before dealing with the actual problem (at static time). typing should be easy and typing failure should be noted as distinct from theorem failure.

In [ ]:
class StagedTheorem():
    stages : list[BoolRef]

    stages : list[dict[BoolRef, BoolRef]]

    dict[ExprRef,ExprRef] # definitions exposed at different stages?




In [ ]:
class StagedCegar():
    static: dict[BoolRef, BoolRef] # mapping of opaque booleans variables b0 abstracting out details in dynamic program
    dynamic: BoolRef  # contains references to b0 

def quote(x : ExprRef):
    x = FreshConst(e.sort())
    ctx[x] = e
    return x

def unquote(x : ExprRef):
    return ctx[x]

pow(x) = quote()

class QuoteExpr():
    def __init__(self, e):
    def __call__(self, *args):
        return quote(self.f(*args))
    def __add__(self, other):
    def __get_attr__(self, name):
        self.e.get_attr(name)
        def res(*args):
            return quote(e(*args))
        return res
    

# Def Eq
We are maintaining a definition thing.



Function("def_eq")?
Obviously too slippery (althoguh it is kind of meaningful if we so happen to hide the definition axioms)
but better is the quoted version. We can kind of reflect definitional equality as an axiom schema into quoted terms.
Function("def_eq", z3_term, z3_term)

Universes and metaprogramming layers feel similar ish. They're both a hiearchy of meta stuff.
Relation to justified smt?



In [ ]:
def defeq(t):
    if in kernel.defns[t.decl()]

# Quote


"oracle"

hilbert epsilon. "choice function". maybe that's more accurate? in the sense a formula is a set?



self mutating sat

continuations. self cegis

Related to the above, we can also build a replication of some of z3's ast inside itself. The `quote` function is a metalevel python notion.

https://arxiv.org/abs/1802.00405
qe hol-light

Quotation is rife with paradox, but I'm not sure that the way I've done it here can be? Since I haven't internalized quote into my logic.


What about having an internalized Formula and solver functions. Similar to formulog but more homoiconic. So it makes sense to have 3 layers or more.
Maybe a way of formulating 

smtlib macro
Homoiconic. Who's to say there isn't a higher level z3 in which we are formulas embedded into. Is there a way to request things of this higher z3?
python3 -m smtpp 
A magic # or something as a tick?

This is also like clingo metaprogramming.

TPTP could for serious use prolog as a preprecossor since it is iu the form of a prolog fact base.


In [ ]:
from z3 import *
#if __name__ == "main":
#    # read string
ast = from_smt2_string("(declare-const x Int) (declare-const n Int) (assert (forall ((x Int) (n Int)) (= (mypow x n) (ite (<= n 0) 1 (* x (mypow x (- n 1)))))))")
# separate out define-macros?
# macros are just functions from syntax to syntax.
# separate out asserts, check-sat, get-model, other commands, declare-const, declare-fun
# define-fun, define-fun-rec, define-const, declare-const, declare-fun, declare-sort, define-sort, declare-datatypes, define-datatypes,

s = Solver()
macroexpand = DefineRecFunction(macroexpand)
runtime_prog = Const("runtime_prog", Code)
s.add(runtime_prog == macroexpand(quote()))
res = s.check()
if res == sat:
    unquote(s.model()[runtime_prog])
# can I do staged asserts?






In [ ]:
BoolExpr = Datatype("BoolExpr")
BoolExpr.declare("BoolVal", ("val", BoolSort())) # splice kind of
BoolExpr.declare("Var", ("name", StringSort()))
BoolExpr.declare("And", ("lhs", BoolExpr), ("rhs", BoolExpr))
BoolExpr.declare("Or", ("lhs", BoolExpr), ("rhs", BoolExpr))
BoolExpr.declare("Not", ("arg", BoolExpr))
BoolExpr.declare("Quote", ("arg", BoolExpr))
BoolExpr = BoolExpr.create()

Model = ArraySort(StringSort(), BoolSort())
sat = Function("sat", BoolExpr, BoolSort()) # sat ~ exists model, interp(form, model)
eps = Function("eps", BoolExpr, Model) # eps ~ model, interp(form, model) = True
satisfies = Function("satisifies", BoolExpr, Model, BoolSort()) # M |= phi   relation.

# satisfies(f, eps(f)) = sat(f)
def cegis_loop(s : Solver):
    res = s.check()
    if = 
    m = s.model()


In [1]:
"""
unityped
Z3Expr = Datatype("Z3Expr")
Z3Expr.declare("IntVal", ("val", IntSort()))
Z3Expr.declare("Var", ("name", StringSort()))
Z3Expr.declare("And", ("lhs", Z3Expr), ("rhs", Z3Expr))
Z3Expr.declare("Or", ("lhs", Z3Expr), ("rhs", Z3Expr))
"""
BoolExpr = Datatype("BoolExpr")
BoolExpr.declare("BoolVal", ("val", BoolSort()))
BoolExpr.declare("Var", ("name", StringSort()))
BoolExpr.declare("And", ("lhs", BoolExpr), ("rhs", BoolExpr))
BoolExpr.declare("Or", ("lhs", BoolExpr), ("rhs", BoolExpr))
BoolExpr.declare("Not", ("arg", BoolExpr))
BoolExpr = BoolExpr.create()
# maybe reals is better
IntExpr = Datatype("IntExpr")
IntExpr.declare("IntVal", ("val", IntSort()))
IntExpr.declare("Var", ("name", StringSort()))
IntExpr.declare("Add", ("lhs", IntExpr), ("rhs", IntExpr))
IntExpr.declare("Sub", ("lhs", IntExpr), ("rhs", IntExpr))
IntExpr.declare("Quote", ("expr", IntExpr)
IntExpr = IntExpr.create()


Result = Datatype("Result")
Result.declare("sat")
Result.declare("unsat")
Result.declare("unknown")
Function("check-sat" BoolSort())

Value = Datatype("Value")
Value.declare("Int", ("int", IntSort()))
Value.declare("Bool", ("bool", BoolSort()))
Value = Value.create()
Model = ArraySort(Id, Value) # Or refactor to (ArraySort(id, Bool), ArraySort(Id, Real))
# Very similar to hilbert epsilon
Function("get-model", BoolExpr, Model)
booleval = Function("booleval", BoolExpr, Model)


def quote(e : ExprRef)-> ExprRef:
    match e.decl().name():
        case "IntVal": return Z3Expr.IntVal(e)
        #case "Var": return Z3Expr.Var(e)
        case "and": return Z3Expr.And(quote(e.arg(0)), quote(e.arg(1)))
        case "or": return Z3Expr.Or(quote(e.arg(0)), quote(e.arg(1)))
        case _: raise Exception(f"Unknown decl: {e.decl().name()}")

# eval = Function(Z3Expr, ArraySort(StringSort(), BoolSort()), BoolSort())
# eval_def = ForAll([e,env], eval(e) == If( ... ))

SyntaxError: '(' was never closed (3140188254.py, line 22)

# User defined propagators
https://microsoft.github.io/z3guide/programming/Example%20Programs/User%20Propagator/

If I have some things I've proven, can I reflect them into z3's mechanisms?

 plant config paper https://web.ist.utl.pt/nuno.lopes/pubs/plantconfig-cpaior.pdf

https://dl.acm.org/doi/abs/10.1007/978-3-031-24950-1_5 sat modulo custom thoeires

But putting regular solving in a loop and looking at models to see if secret constraints are violated is just as viable? It may not be performant.


Refinement typing via user propagator

https://link.springer.com/chapter/10.1007/978-3-031-43513-3_2 Non-Classical Logics in Satisfiability Modulo Theories https://github.com/CEisenhofer/ModalZ3


# Pattern matching on z3 asts


In [13]:
from typing import Optional, Iterable, Dict
def match_(t : AstRef, pat : AstRef, vars : Iterable[ExprRef] = []) -> Optional[Dict[ExprRef,ExprRef]]:
    subst = {}
    todo = [(t,pat)]
    while todo:
        t,pat = todo.pop()
        if pat in vars or is_var(pat): # allow var as pattern?
            if pat in subst:
                if subst[pat].eq(t):
                    pass
                else:
                    return None
            else:
                subst[pat] = t
        elif isinstance(t, QuantifierRef) or isinstance(pat, QuantifierRef):
            raise NotImplementedError
        else:
            thead, targs = t.decl(), t.children()
            phead, pargs = pat.decl(), pat.children()
            if thead != phead or len(targs) != len(pargs): # check sorts here? The decl might check that
                return None
            todo.extend(zip(targs, pargs))
    return subst

E = DeclareSort("Expr")
foo = Function("foo", E, E, E)
bar = Function("bar", E, E)
a,b,c = Consts("a b c", E)
x,y,z = Consts("x y z", E)
vars = {x,y,z}

assert match_(foo(x, bar(y)), foo(a, bar(b)), vars) == None
assert match_(foo(a, bar(b)), foo(x, bar(y)), vars) == {x:a, y:b}
assert match_(foo(a, bar(b)), foo(a, bar(b)), vars) == {}
assert match_(foo(a, bar(b)), foo(x,x), vars) == None
assert match_(foo(a, bar(a)), foo(x,bar(x)), vars) == {x:a}
match_(foo(a, bar(a)), Lambda([x],foo(x,bar(x))).body()) == {Var(0, E):a}


True

matching as an internalized construct.

Note in my 2019 blog post I used an interesting keyword arg based desing


In [ ]:
z3.DatatypeRef.match = lambda self, **kwargs: 

In [8]:
Var(0, IntSort())

AttributeError: 'ArithRef' object has no attribute 'body'

# Proof Display

In [8]:
from IPython.core.display import display, HTML
display(HTML('<div width="100px"><hr></div>&forall;x'))

/tmp/ipykernel_1070664/3665299815.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [26]:
from IPython.core.display import HTML

html_content = """
<style>
  .proof-tree {
    font-family: 'DejaVu Sans', Arial, sans-serif;
    line-height: 1.5;
    color: white; /* Set text color to white for dark background */
  }
  .rule {
    display: flex;
    flex-direction: column;
    align-items: center;
    margin-bottom: 20px;
  }
  .premises {
    display: flex;
    justify-content: bottom;
    align-items: flex-end; 
  }
  .premises div {
    margin: 0 10px;
  }
  .conclusion {
    border-top: 1px solid white; /* Set border color to white for visibility */
    padding-top: 5px;
    text-align: bottom;
  }
</style>

<div class="proof-tree">

  <div class="rule">
    <div class="premises">
        <div class="rule">
        <div class="premises">
           <div class="conclusion">A</div>
           <div class="conclusion">B</div>
       </div>
       <div class="conclusion">A &#x2227; B</div>
    </div>
      <div class="conclusion">C</div>
    </div>
    <div class="conclusion">(A &#x2227; B) &#x2227; C</div>
  </div>
</div>
"""

HTML(html_content)


class Proof():
    def tree(self, depth=3):
        if depth <= 0:
            return "<div>...</div>"
        return f"""
        <div class="premises">
          {"\n".join(r.tree(depth-1) for r in self.reasons)}
        </div>
        <div class="conclusion">{repr(self.thm)}</div>
        """



# Python 

## typing
My monkeypatching is not a great solution. It's janky. 

A registry connecting sorts to classes


in z3py AstRef has __init__
it takes ast reference and ctx

cvc5 has ast reference and context in ExprRef, there is no ASTRef

Using registries means python will never infer types well.
I could put in a smart if then else for my most important builtins and leave the rest up to the registry

if sort() == IntSort()

If I were to just grab the pieces that would not be chill.

What did Saul do?
I could make kd.Record a decorator.

classmethod

@kd.Record

@kd.Record
class Foo:
    x: Int (?)
    y: Int

This is a whole shithole I'm going down. Why am I doing it?


In [32]:
import knuckledragger as kd

kd.kernel.Proof["Hello world"]

TypeError: 'NoneType' object is not subscriptable

In [ ]:

class ExprRef(ExprRef):
    """Only for improved type checking. Need more full rewrapping of FuncDecl etc if we really want to go this way."""
    def __add__(self, other):
        pass

In [ ]:
# I guess os could go tyhe other way or be one-to-many, many to many?
# This seems natrual though
#sort_registry : dict[SortRef, type] = {}

# hmm it's not quite the same, because I want the wrap to return the wrapped value
class_ : Callable[ExprRef, type] = kd.notation.SortDispatch() #$ returns class
def wrap(x : ExprRef) -> class_[x.sort()]:
    x.__class__ = class_(x)
    return x
ExprRef.wrap = wrap

old_Const = Const
def Const(name, sort):
    x = old_Const(name, sort)
    if sort in class_:
        return wrap(x)
    else:
        return x


### pyi file
https://mypy.readthedocs.io/en/stable/stubs.html


In [ ]:
import knuckledragger as kd

class AstRef: ...

class ExprRef(AstRef):
    def __add__(self, other: ExprRef) -> ExprRef: ...
    def __radd__(self, other: ExprRef) -> ExprRef: ...
    def __sub__(self, other: ExprRef) -> ExprRef: ...

class FuncDeclRef:
    defn: kd.kernel.Proof


In [ ]:
class MyRecordSort(DatatypeSortRef):

class MyRecord(DatatypeRef):
    def __init__(self, name):
        self = Const(name, MyRecordSort)
    @property
    def x(self):
        return MyRecordSort.x(self)
class MyRecordVal(DatatypeRef):
    def __init__(self, *args):
        self = MyRecordSort(*args)
    @property
    def x(self):
        return MyRecordSort.x(self)

def Const(name, sort : SortRef):


In [ ]:
@dataclass(frozen=True)
class Nat(DatatypeRef):
    wf_pf : kd.Proof
# but the proof is usually contextual?


## python syntax / applicative python
https://docs.python.org/3/library/codecs.html#codecs.register
https://docs.python.org/3/reference/import.html
https://news.ycombinator.com/item?id=41322758 python's preprocessor. Fiendish macros.

I have seen people do special stuff in jupyter. I don't rerally liek any of that

Nelli
Ast. Rewrite if elif else -> If
rewrite return to null?
Overloading kind of works, except 
and or not to And Or Not
If
rewrite x <= y <= z


could convert ifthenelse to `x if c else`.
Nah.

chococpy
codon
mypyc


buildit style



In [1]:
class Bool():
    def __bool__(self):
        print("asked")
        return True
    

b = Bool()
if b:
    print("yes")
else:
    print("no")

import z3

ctx = None
path = []
s = Solver()
def record_bool():
    s.check()
    path.append()

z3.BoolRef.__bool__ = lambda self, 
def reify(f, vs):
    global ctx
    ctx = [m]
    while True:
        retval = f(vs)
        
    ctx = None

@reify
def example(x):
    if x == 7:
        return 3
    else:
        if x > 14:
            return 4
        else:
            return 5


asked
yes


# Lark notation

https://claude.ai/chat/6ddab7ff-d1ba-40d2-9b9c-1a64b5cebd71

Hmm. I really need type inference to make this pleasant
foo == bar makes it hard to know what foo is.

What if I used the python env.
Any constqnt is just looked up in the caller's env

lark has a python grammar.

I kind of want python, but with different function call syntax and implices, forall, exists.
https://github.com/lark-parser/lark/blob/master/lark/grammars/python.lark


In [26]:
from lark import Lark, Transformer, v_args
pyexpr = """

// quant: "forall" bvar "," quant -> forall | "exists" name+ "," quant -> exists | "lambda" name+ ":" quant -> lambda | test
// bvar: "(" name+  ":" name ")" -> type_annot | name -> var
?test: or_test ("if" or_test "else" test)?

?or_test: and_test ("or" and_test)*
?and_test: not_test_ ("and" not_test_)*
?not_test_: "not" not_test_ -> not_test
         | comparison
?comparison: expr (comp_op expr)*

?expr: or_expr
?or_expr: xor_expr ("|" xor_expr)*
?xor_expr: and_expr ("^" and_expr)*
?and_expr: shift_expr ("&" shift_expr)*
?shift_expr: arith_expr (_shift_op arith_expr)*
?arith_expr: term (_add_op term)*
?term: factor (_mul_op factor)*
?factor: _unary_op factor | power


!_unary_op: "+"|"-"|"~"
!_add_op: "+"|"-"
!_shift_op: "<<"|">>"
!_mul_op: "*"|"@"|"/"|"%"|"//"
// <> isn't actually a valid comparison operator in Python. It's here for the
// sake of a __future__ import described in PEP 401 (which really works :-)
!comp_op: "<"|">"|"=="|">="|"<="|"<>"|"!="|"in"|"not" "in"|"is"|"is" "not"

?power: await_expr ("**" factor)?
?await_expr: atom_expr

?atom_expr:  atom+ -> funccall
// atom_expr "(" [arguments] ")"      -> funccall
//          | atom_expr "[" subscriptlist "]"  -> getitem
//          | atom_expr "." name               -> getattr
//          | atom

?atom: 
     | name -> var
     | number
     | "(" test ")"
     | "True"    -> const_true
     | "False"   -> const_false
          
!name: NAME | "match" | "case"
NAME: /[^\W\d]\w*/

number: DEC_NUMBER | HEX_NUMBER | BIN_NUMBER | OCT_NUMBER | FLOAT_NUMBER | IMAG_NUMBER
// string: STRING 

//STRING: /([ubf]?r?|r[ubf])("(?!"").*?(?<!\\)(\\\\)*?"|'(?!'').*?(?<!\\)(\\\\)*?')/i

_SPECIAL_DEC: "0".."9"        ("_"?  "0".."9"                       )*
DEC_NUMBER:   "1".."9"        ("_"?  "0".."9"                       )*
          |   "0"             ("_"?  "0"                            )* /(?![1-9])/
HEX_NUMBER.2: "0" ("x" | "X") ("_"? ("0".."9" | "a".."f" | "A".."F"))+
OCT_NUMBER.2: "0" ("o" | "O") ("_"?  "0".."7"                       )+
BIN_NUMBER.2: "0" ("b" | "B") ("_"?  "0".."1"                       )+

_EXP: ("e"|"E") ["+" | "-"] _SPECIAL_DEC
DECIMAL: "." _SPECIAL_DEC | _SPECIAL_DEC "." _SPECIAL_DEC?
FLOAT_NUMBER.2: _SPECIAL_DEC _EXP | DECIMAL _EXP?
IMAG_NUMBER.2: (_SPECIAL_DEC      | FLOAT_NUMBER) ("J" | "j")
"""

Lark(pyexpr, start="test", parser="lalr").parse("x + 1")

GrammarError: Reduce/Reduce collision in Terminal('__ANON_6') between the following rules: 
	- <atom : >
	- <atom_expr : __atom_expr_plus_9>

Reduce/Reduce collision in Terminal('$END') between the following rules: 
	- <atom : >
	- <atom_expr : __atom_expr_plus_9>

Reduce/Reduce collision in Terminal('OR') between the following rules: 
	- <atom : >
	- <atom_expr : __atom_expr_plus_9>

Reduce/Reduce collision in Terminal('IF') between the following rules: 
	- <atom : >
	- <atom_expr : __atom_expr_plus_9>

Reduce/Reduce collision in Terminal('IS') between the following rules: 
	- <atom : >
	- <atom_expr : __atom_expr_plus_9>

Reduce/Reduce collision in Terminal('PERCENT') between the following rules: 
	- <atom : >
	- <atom_expr : __atom_expr_plus_9>

Reduce/Reduce collision in Terminal('IN') between the following rules: 
	- <atom : >
	- <atom_expr : __atom_expr_plus_9>

Reduce/Reduce collision in Terminal('__ANON_0') between the following rules: 
	- <atom : >
	- <atom_expr : __atom_expr_

In [20]:
from lark import Lark, Transformer, v_args
pyexpr = """


?test: or_test ("if" or_test "else" test)?
//     | lambdef
     | assign_expr

assign_expr: name ":=" test

// ?test_nocond: or_test | lambdef_nocond

?or_test: and_test ("or" and_test)*
?and_test: not_test_ ("and" not_test_)*
?not_test_: "not" not_test_ -> not_test
         | comparison
?comparison: expr (comp_op expr)*

?expr: or_expr
?or_expr: xor_expr ("|" xor_expr)*
?xor_expr: and_expr ("^" and_expr)*
?and_expr: shift_expr ("&" shift_expr)*
?shift_expr: arith_expr (_shift_op arith_expr)*
?arith_expr: term (_add_op term)*
?term: factor (_mul_op factor)*
?factor: _unary_op factor | power

!_unary_op: "+"|"-"|"~"
!_add_op: "+"|"-"
!_shift_op: "<<"|">>"
!_mul_op: "*"|"@"|"/"|"%"|"//"
// <> isn't actually a valid comparison operator in Python. It's here for the
// sake of a __future__ import described in PEP 401 (which really works :-)
!comp_op: "<"|">"|"=="|">="|"<="|"<>"|"!="|"in"|"not" "in"|"is"|"is" "not"

?power: await_expr ("**" factor)?
?await_expr: AWAIT? atom_expr
AWAIT: "await"

?atom_expr:  atom+ -> funccall
//atom_expr "(" [arguments] ")"      -> funccall
//         | atom_expr "[" subscriptlist "]"  -> getitem
//          | atom_expr "." name               -> getattr
//          | atom

?atom: 
// "(" yield_expr ")"
//     | "(" _tuple_inner? ")" -> tuple
//     | "(" comprehension{test_or_star_expr} ")" -> tuple_comprehension
//     | "[" _exprlist? "]"  -> list
//     | "[" comprehension{test_or_star_expr} "]"  -> list_comprehension
//     | "{" _dict_exprlist? "}" -> dict
//     | "{" comprehension{key_value} "}" -> dict_comprehension
//     | "{" _exprlist "}" -> set
//     | "{" comprehension{test} "}" -> set_comprehension
     | name -> var
     | number
//     | string_concat
     | "(" test ")"
     | "..." -> ellipsis
     | "None"    -> const_none
     | "True"    -> const_true
     | "False"   -> const_false

?subscriptlist: subscript
              | subscript (("," subscript)+ [","] | ",") -> subscript_tuple
?subscript: test | ([test] ":" [test] [sliceop]) -> slice
     
arguments: argvalue ("," argvalue)*  ("," [ starargs | kwargs])?
    | starargs
    | kwargs
    | comprehension{test}

!name: NAME | "match" | "case"
NAME: /[^\W\d]\w*/

number: DEC_NUMBER | HEX_NUMBER | BIN_NUMBER | OCT_NUMBER | FLOAT_NUMBER | IMAG_NUMBER
string: STRING | LONG_STRING

_SPECIAL_DEC: "0".."9"        ("_"?  "0".."9"                       )*
DEC_NUMBER:   "1".."9"        ("_"?  "0".."9"                       )*
          |   "0"             ("_"?  "0"                            )* /(?![1-9])/
HEX_NUMBER.2: "0" ("x" | "X") ("_"? ("0".."9" | "a".."f" | "A".."F"))+
OCT_NUMBER.2: "0" ("o" | "O") ("_"?  "0".."7"                       )+
BIN_NUMBER.2: "0" ("b" | "B") ("_"?  "0".."1"                       )+

_EXP: ("e"|"E") ["+" | "-"] _SPECIAL_DEC
DECIMAL: "." _SPECIAL_DEC | _SPECIAL_DEC "." _SPECIAL_DEC?
FLOAT_NUMBER.2: _SPECIAL_DEC _EXP | DECIMAL _EXP?
IMAG_NUMBER.2: (_SPECIAL_DEC      | FLOAT_NUMBER) ("J" | "j")

"""

Lark(pyexpr).parse("x + 1")

GrammarError: Rule 'sliceop' used but not defined (in rule subscript)

In [ ]:

logic = """
?expr:  expr*  | "(" expr ")" | expr binop expr | 
?formula: forall idens* "," formula | exists idens* "," formula | "fun" idens* "=>" formula | expr   
?iden: CNAME | CNAME ":" CNAME
?unop: "-" | "~"
?binop: "+" | "-" | "*" | "/" | "<" | ">" | "=" | "!=" | "<=" | ">=" | "and" | "or"


"""


In [30]:
import lark

basic_logic = """
?start: formula
?formula: "true" | "false" | "not" formula | formula "and" formula | formula "or" formula | "(" formula ")"
?not: "not" | "!" | "~"
?and: "and" | "&" | "&&" | "/\\"
?or: "or" | "|" | "||" | "\\/"
"""

expr = """

"""

set_logic = """
?builder: "{" expr "|" formula "}"
"""

sum_expr = """

"""

parser = lark.Lark(basic_logic)
parser.parse("true and false")

UnexpectedCharacters: No terminal matches ' ' in the current parser context, at line 1 col 5

true and false
    ^
Expected one of: 
	* OR
	* AND


# Well Formed
check()
## Inductives

## Refinement
projection

## Quotient
canon

## Coinductives

# Interfaces / Algerbaic Hierarchy

ABS abstract base classes vs Protocol

`Proof["forall x, x >= 0"]` or `Proof[ForAll([x], x >= 0)]`

can I tag proof objects with what they should kind of contain? Nice for interfaces

https://www.why3.org/stdlib/algebra.html


In [1]:
# this is a metalevel depdent record.
@dataclass(frozen=True)
class Group():
    G : SortRef
    e : ConstRef
    mul : FuncDeclRef
    inv : FuncDeclRef
    assoc : Proof
    inv_unit : Proof
    unit : Proof
    def check(self):
        kd.lemma(ForAll([x], mul(x, inv(x)) == e), by=[unit])
        kd.lemma(ForAll([x], mul(x, e) == x), by=[unit])
    def __post_init(self):
        self.check()
# metalevel record creation
#def Group_G(G):
#   kd.Record("Group){G}", (G, e, , mul))
#   notation.check()

def GroupLemmas(G : Group):
    pass


G = DeclareSort("G")
e = Const("e", G)
mul = Function("mul", G, G, G)
inv = Function("inv", G, G)
assoc = kd.axiom(ForAll([x,y,z], mul(mul(x,y),z) == mul(x,mul(y,z))))
inv = kd.axiom(ForAll([x], mul(x, inv(x)) == e))
unit = kd.axiom(ForAll([x], mul(x,e) == x))

# free group over what?
FreeGroup = Group(G, e, mul, inv, assoc, inv_unit, unit)
GroupLemmas(FreeGroup)

Set(G)

@dataclass(frozen=True)
class Ring():
    G : Group
    def check(self):
        G.G == self.T


NameError: name 'dataclass' is not defined

In [ ]:
find . -type f -name 'f*'

In [ ]:
class R():
    T = RealSort()
    add = kd.define("add", T, T, T)
    mul = kd.define("mul", T, T, T)

#vs
class R(ExprRef):

#vs
class R(SortRef)
    



In [ ]:
class RMonoid(algebra.Monoid):
    munit = 



RMonoid = Monoid(munit=RealVal(0),
            mplus=add, iden=add_zero, assoc=add_assoc)

@dataclass
class RRing(algebra.Ring):



In [ ]:
def Metric(Protocol):
    T : SortRef
    dist : FuncDeclRef | ArrayRef
    def dist(x,y):
        pass
    pos = None
    sym = None
    refl = None
    triangle = None

Modules in ocaml let's say are records that contain values and types.
It is not an issue since we are in a metalanguage to have sorts in our records.
dataclasses are reasonable.

They also should contain proofs.
Kind of dependent records.
Compare with isabelle locales.


In [ ]:
from z3 import *
from dataclasses import dataclass

# since we don't have polymorphism, we need to do module style for any container.
@dataclass(frozen=True)
class OrderedType():
    t : SortRef
    le : FuncDeclRef
    le_total : BoolRef # Proof
__OrderedType = OrderedType
def OrderedType(elem : ):
    return __OrderedType(elem[0], elem[1], elem[2])
# https://v2.ocaml.org/api/Set.Make.html
@dataclass(frozen=True)
class SetModule():
    t : SortRef
    elt: SortRef
    cons : FuncDeclRef
    nil : FuncDeclRef
    head : FuncDeclRef
__SetModule = SetModule
# smart constructor
def SetModule(elem : ):

In [ ]:
def monoid(S):
    return {
        "set": S,
        "op": Function("op", S, S, S),
        "unit": Const("unit", S),
        "assoc": ForAll([x,y,z], op(op(x,y),z) == op(x,op(y,z))),
        "unitl": ForAll([x], op(unit,x) == x),
        "unitr": ForAll([x], op(x,unit) == x),
    }    

def monoid(S,op,unit, hyps):
    assoc = ForAll([x,y,z], op(op(x,y),z) == op(x,op(y,z)))
    prove(assoc)
    unitl = ForAll([x], op(unit,x) == x)
    prove(unitl)
    unitr = ForAll([x], op(x,unit) == x)
    prove(unitr)
    return {
        "set": S,
        "op": op,
        "unit": unit,
        "assoc": assoc,
        "unitl": unitl,
        "unitr": unitr,
    }

# check parametric stuff on fresh sorts, but then can trust.

A Section or locale or module mechanims is pretty similar to an Isar proof. We tend t make things stored whereas proofs have anonymus

```python
class Section(): # locale?
    def _init__(self, name, type_params=[A]):
        self.fixes = []
        self.assumes = []
        self.defs = []
    def fix(self, x):
        self.fixes.append(x)
        return self
    def assume(self, x):
        self.assumes.append(x)
        return self
    def define(self, x):
        self.defs.append(x)
        return self
    def instance(self, fixes,assumes):
    def sublocale(self):
        # extend a locale


lattice = Section("lattice")\
    .fixes("x", )\
    .fixes("meet", ArraySort())\
    .fixes("join", )\
    

```

<https://isabelle.in.tum.de/library/HOL/HOL/Orderings.html>
Is a decent root
locale partial_preordering =
  fixes less_eq :: ‹'a ⇒ 'a ⇒ bool› (infix ‹❙≤› 50)
  assumes refl: ‹a ❙≤ a› ― ‹not ‹iff›: makes problems due to multiple (dual) interpretations›
    and trans: ‹a ❙≤ b ⟹ b ❙≤ c ⟹ a ❙≤ c›

locale preordering = partial_preordering +
  fixes less :: ‹'a ⇒ 'a ⇒ bool› (infix ‹❙<› 50)
  assumes strict_iff_not: ‹a ❙< b ⟷ a ❙≤ b ∧ ¬ b ❙≤ a›
begin

Hmm. Obvously isabelle uses it's parametricity.
I guess the only thing to do is use meta parametricity.
Could instantiate with sort to sanity check def and then run all proofs at instnatiation. Or could elide checking process if we _really_ trust this meta reasoning

There's just so much boiler plate. Ugh

Another option is to pick a universal sort. And have injections in and out

```python
# locales as functions. But how do we sanity check at definition time?
def partial_ordering(A,leq,axioms):
    x = FreshConst(A, prefix="x")
    refl = infer( axioms, ForAll([x], leq(x,x)))

def partial_ordering(sorts, sig, axioms):
    A = sorts["A"]
    leq = sig["leq"]
    x = FreshConst(sorts["A"], prefix="x")
    refl = infer()

def partial_ordering(ctx):
    x = FreshConst(ctx.A, prefix="x")
    refl = infer( axioms, ForAll([x], leq(x,x)))

class Partial_Ordering():
    sig = # fixes 
    axioms = # assumes
    def instan(self):
        pass

```

```python
# Can we embed shallower into python class system?
# it becomes a pattern rather than a library?
# Ughh. I just know this is getting hairy and silly for no reason
class Locale():
    def __init__(cls,name,bases,clsdict) -> None:
        super().__init__(name,bases,clsdict)
        cls.fixes = []
        cls.assumes = []
        cls.defs = []

# use decorator
@locale
class Partial_Preordering(metaclass=Locale): # metaclass is so funky.
    leq = Function("leq",A,A,BoolSort())
    fixes = [Function("leq", A, A, BoolSort())]
    assumes = {"refl":   }
    def __init__(self, A, leq):
        leq = Function("leq",A,A,BoolSort())
        fixes = [Function("leq", A, A, BoolSort())]
        assumes = {"refl":   }

# is initializing the class the same and intepreting the locale?
# How 
class Preordering(Partial_Preordering):
    def __init__(self, A, leq, ):
        assert leq.sort() == ArraySort(A,A,BoolSort())


```

```python
z3.simplify(IntVal(3) / 2)
```

1

# Any / Generics / SMTLib3
Maybe "Any" is ~ ZF set.

https://github.com/Z3Prover/z3/blob/11218154391d5137ab35a3281fb6283587c08ba0/src/api/python/z3/z3.py#L713 z3 apparently has alpha support for type variables


First kind of generics is top use python metaprogramming.
`lambda T: Lambda([Const("x", T)], Const("x", T))`    is the same as the system T term  `Lam T. \x : T. x`

Internalized generics are interesting though.

Can we make a shallow embedding of generic types or 

https://smt-lib.org/version3.shtml


We might want a typechecker running at the python layer. But typechecking is an approximation to some semantic property of interest? So maybe we could directly check that

Internalize modules.



In [212]:
Any0 = DeclareSort("KD_Any0")
Any1 = DeclareSort("KD_Any1")
Any2 = DeclareSort("KD_Any2")
Anys = [Any0, Any1, Any2]

def instantiate_sorts_decl(f : z3.FuncDecl, sort_subst) -> z3.FuncDecl:
    assert all(k in Anys for k in sort_subst.keys())
    return Function(f.name(), *[sort_subst.get(f.domain(i), f.domain(i)) for i in range(f.arity())], sort_subst.get(f.range(), f.range()))

def instantiate_sorts_expr(t : z3.ExprRef, sort_subst):
    #assert kd.kernel.is_proof(thm)
    #assert all(k in Anys for k in sort_subst.keys())
    if z3.is_app(t):
        decl = instantiate_sorts_decl(t.decl(), sort_subst)
        return decl(*[instantiate_sorts_expr(a, sort_subst) for a in t.children()])
    elif z3.is_quantifier(t):
        if t.is_forall():
            return z3.ForAll([instantiate_sorts_expr(v, sort_subst) for v in t.children()], instantiate_sorts_expr(t.body(), sort_subst))
        else:
            raise NotImplementedError
    elif z3.is_var(t):
        return z3.Var(z3.get_var_index(t), sort_subst.get(t.sort(), t.sort()))
    else:
        raise NotImplementedError

def instantiate_sorts_schema(pf : kd.kernel.Proof, *sort_subst) -> kd.kernel.Proof:
    assert kd.kernel.is_proof(pf)
    sort_subst = {k:v for k,v in sort_subst}
    assert all(k in Anys for k in sort_subst.keys())
    return kd.axiom(instantiate_sorts_expr(pf.thm, sort_subst), reasons=[p, sort_subst])  

x,y,z = z3.Consts("x y z", Any0)
#id_ = Function("id", Any0, Any0)
#kd.axiom(z3.ForAll([x], id_(x) == x))
id_ = kd.define("id", [x], x)
id_int = instantiate_decl(id_, {Any0: IntSort()})
#dir(id_)
id_int
id_int_def = instantiate_sorts_schema(id_.defn, (Any0, IntSort()))


Z3Exception: Invalid bounded variable(s)

In [44]:
%%file /tmp/test.smt2
(declare-datatypes () ((MyT (Nil) (MyConstr (v Bool) (hd (Array Bool MyT))))))
(declare-const x MyT)
(check-sat)
(get-model)

Overwriting /tmp/test.smt2


In [59]:
!z3 /tmp/test.smt2

^C
unknown
(error "line 7 column 10: model is not available")


In [58]:
%%file /tmp/test.smt2
(declare-datatypes ((MyT 0)) (((Nil) 
                               (MyConstr (hd (Array Int MyT))))))
(declare-const x MyT)
(assert (distinct x Nil))
;(assert (distinct (select (hd x) 7) Nil))
(check-sat)
(get-model)

Overwriting /tmp/test.smt2


In [76]:

import z3
i = z3.Int('i')
g = z3.Goal()
g.add(i > 2)
g.add(i < 4)
z3.Then('simplify', 'propagate-ineqs').apply(g)

[[i == 3]]

In [29]:
%%file /tmp/test3.smt2
(declare-sort Expr)
(declare-sort Var)
(declare-datatypes ((Stmt 0)) 
  (((Assignment (lval Var) (rval Expr)) 
    (If (cond Expr) (th Stmt) (el Stmt)) 
    (Seq (stmts (Seq Stmt))))))

(declare-const s Stmt)
(declare-const t Stmt)

(assert ((_ is Seq) t))
(assert ((_ is Seq) s))
(assert (= s (seq.nth (stmts t) 2)))
(assert (>= (seq.len (stmts s)) 5))
(check-sat)
(get-model)
(assert (= s (Seq (seq.unit s))))
(check-sat)

Writing /tmp/test3.smt2


In [30]:
! z3 /tmp/test3.smt2

sat
(
  ;; universe for Expr:
  ;;   Expr!val!7 Expr!val!3 Expr!val!1 Expr!val!4 Expr!val!0 Expr!val!6 Expr!val!5 Expr!val!2 
  ;; -----------
  ;; definitions for universe elements:
  (declare-fun Expr!val!7 () Expr)
  (declare-fun Expr!val!3 () Expr)
  (declare-fun Expr!val!1 () Expr)
  (declare-fun Expr!val!4 () Expr)
  (declare-fun Expr!val!0 () Expr)
  (declare-fun Expr!val!6 () Expr)
  (declare-fun Expr!val!5 () Expr)
  (declare-fun Expr!val!2 () Expr)
  ;; cardinality constraint:
  (forall ((x Expr))
          (or (= x Expr!val!7)
              (= x Expr!val!3)
              (= x Expr!val!1)
              (= x Expr!val!4)
              (= x Expr!val!0)
              (= x Expr!val!6)
              (= x Expr!val!5)
              (= x Expr!val!2)))
  ;; -----------
  ;; universe for Var:
  ;;   Var!val!2 Var!val!0 Var!val!3 Var!val!4 Var!val!5 Var!val!6 Var!val!7 Var!val!1 
  ;; -----------
  ;; definitions for universe elements:
  (declare-fun Var!val!2 () Var)
  (declare-fun Var!

(error "line 1 column 63: unknown sort 'MyT'")
(error "line 2 column 17: Invalid constant declaration: unknown sort 'MyT'")
sat
(
)


a typeclass system?
Sort level unification. unify up0on application

class GenericDecl(z3.FuncDecl):
    def __call__(self, *args):
        unify(self.domain(), )
        unify(self.range(),  )
        f = instantiate_decl(self, subst)
        args = instantiate_args(args, subst)
        return f(*args)
class 

def TVar(prefix=""):
    counter += 1
    return DeclareSort("prefix + "_KDGEN" +str(counter))

Backtick or some other sigil? # ` ' __

Yeah, that tvars are alpha invariant can lead to some goody behavior in principle.
I think it's not thta big a deal. You muight accidentally be less generic than you thought.
Gensyming them might help a little


def TLambda(ts, body):
def TLambda(n, f):
    f(*[KDGernetic(i) for i in range(n)]


Parametricity theorems


In [ ]:
def unify_sorts(s : SortRef, t : SortRef):
    todo = [(s,t)]
    while len(todo) > 0
        s,t = todo.pop()
        if s == t:
            continue
        if s in Anys:
            if s == t:
                pass
            else:
                return None

        if s.kind() == z3.Z3_INT_SORT and t.kind() == z3.Z3_INT_SORT:
            return {}
        if s.kind() == z3.Z3_BOOL_SORT and t.kind() == z3.Z3_BOOL_SORT:
            return {}
        if s.kind() == z3.Z3_ARRAY_SORT and t.kind() == z3.Z3_ARRAY_SORT:
            return unify_sorts(s.domain(), t.domain()) + unify_sorts(s.range(), t.range())
        if s.kind() == z3.Z3_DATATYPE_SORT and t.kind() == z3.Z3_DATATYPE_SORT:
            return {}
        if s.kind() == z3.Z3_UNINTERPRETED_SORT and t.kind() == z3.Z3_UNINTERPRETED_SORT:
            return {s:t}
        return None

In [193]:
v = z3.Var(0, IntSort())
v1 = z3.Var(1, IntSort())
dir(v)

['__add__',
 '__and__',
 '__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_repr_html_',
 'arg',
 'as_ast',
 'ast',
 'children',
 'ctx',
 'ctx_ref',
 'decl',
 'eq',
 'from_string',
 'get_id',
 'hash',
 'is_int',
 'is_real',
 'num_args',
 'params',
 'serialize',
 'sexpr',
 'sort',
 'sort_kind',
 'tptp',
 'translate',
 'use_pp']

# z3 tvar


In [37]:
from z3 import *

T = DeclareTypeVar("T")
f = Function("id", T, T)
x = Const("x", T)
t = ForAll([x], f(x) == x)
y = Int("y")
f(y).sort()
z = Real("z")
f(z).sort()
s = Solver()
s.add(t)
s.add(f(z) != z)
s.check()
s.model()
ArraySort(IntSort(), IntSort()).subsort(ArraySort(T,T))

from z3 import *
T = DeclareTypeVar("T")
List = Datatype("List")
List.declare("nil")
List.declare("cons", ("car", T), ("cdr", List))
List = List.create()
ilist = List.cons(IntVal(3), List.nil)
rlist = List.cons(RealVal(3), List.nil)
assert ilist.sort() == rlist.sort() # same sort
assert not ilist.eq(rlist)
s = Solver()
s.add(ilist == rlist)
assert s.check() == unsat

assert List.car(ilist).sort() == T
assert List.car(rlist).sort() == T

# fails to even type check... ok.
# s = Solver()
#s.add(List.car(ilist) == IntVal(3))
#s.check()

s = Solver()
s.add(ilist ==  List.cons(IntVal(1) + IntVal(2), List.nil))
s.check()
f(z).sort()

# once applied, it becomes a different thing.
f(z).decl() == f
f(x).decl()(z)


id(z)

Even if I made a List type using this, the induction principle and others would not be ergonomic.

### Closed Any
closed any is a bit more like ACL2 in the sense there is thiis global big honker dataytype
isomorphism from/to uit
Maybe more like GHC generics? Tuple, Sum, 
extensible variants by keeping a registry of what tags we've handed out?

Base = Real | Int | String | Bool | BitVec8-64 | 

type Any = 
| Base of Base
| Tag(Int, AnyC)
| Tuple(AnyC, AnyC)
| Arrayow(Base, Any)
| Seq(Any)



In [ ]:
from z3 import *
Any = DeclareSort("Any") # AnyO. Any Open

anys = {}
def register(T):
    # maybe just memoize?
    assert T not in anys
    inj = Function("inj", T, Any)
    proj = Function("proj", Any, T)
    ax = kd.axiom(ForAll([x], proj(inj(x)) == x))
    # could have disjointness axioms, but don't need them.
    anys[T] = inj, proj, ax
    return inj, proj, ax

to_any = SortDispatch(name="to_any")
ExprRef.to_any = lambda self: anys[self.sort()][0](self)
ExprRef.to_real.register(Any, Function("proj", Any, RealSort()))
ExprRef.to_int.register(Any, Function("proj", Any, IntSort()))
ExprRef.to_bool.register(Any, Function("proj", Any, BoolSort()))
ExprRef.to_string.register(Any, Function("proj", Any, StringSort()))
ExprRef.to_seq.register(Any, Function("proj", Any, SeqSort(Any)))
ExprRef.to_set.register(Any, Function("proj", Any, SetSort(Any)))



# closed Any
# is a rich any more useful than a minimal any?
AnyC = Datatype("AnyC")
AnyC.declare("nil")
AnyC.declare("Bool", ("bool", BoolSort()))
AnyC.declare("Int", ("int", IntSort()))
AnyC.declare("Real", ("real", RealSort()))
AnyC.declare("String", ("string", StringSort()))
#AnyC.declare("Seq", ("seq", SeqSort(AnyC)))
#AnyC.declare("Set", ("set", SetSort(AnyC))) # actually, does this way lie unsoundness? I think z3 will stop me... This is ok?
AnyC.declare("AnySeq", ("anyseq", ArraySort(IntSort(), AnyC))) # This should be logically ok. But it doesn't fly because of some error.
AnyC.declare("RSeq", ("rseq", ArraySort(IntSort(), RealSort())))
AnyC.declare("RSet", ("rset", ArraySort(RealSort(), BoolSort())))
AnyC.declare("pair", ("fst", AnyC), ("snd", AnyC))
AnyC = AnyC.create()


ExprRef.to_anyc.register(RealSort(), AnyC.Real) # = lambda self: anys[self.sort()][0](self)
ExprRef.to_real.register(AnyC, AnyC.real)

ExprRef.to_int.register(RealSort(), z3.ToInt)
ExprRef.to_real.register(IntSort(), z3.ToReal)

"""
and so on.
ExprRef.to_int.register(Any, Function("proj", Any, IntSort()))
ExprRef.to_bool.register(Any, Function("proj", Any, BoolSort()))
ExprRef.to_string.register(Any, Function("proj", Any, StringSort()))
ExprRef.to_seq.register(Any, Function("proj", Any, SeqSort(Any)))
ExprRef.to_set.register(Any, Function("proj", Any, SetSort(Any)))
"""

"""
Set = Datatype("Set")
Set.declare("mk", ("set", SetSort(Set))) 
Set = Set.create()
"""


In [ ]:
from z3 import *
#open any type. Which is more of interest
Any = DeclareSort("Any") # Value? Term?
Type = DeclareSort("Sort")
T, U = Consts("T U", Sort)
Lambda([T, x], annot(x, T)) # hmm. no way to express x is a T.

annot = Function("annot", Any, Sort, Any)
ofbool = Function("ofbool", BoolSort(), Any)


BitVec = Function("BitVec", IntSort(), Sort)
Lambda([x], BitVec(annot(x)))


# but we can also collect at the end a closed form of any


# Parametricity becomes a theorem

In [11]:
OrderMod = Datatype("OrderMod")
OrderMod.declare("ORDMAKE", ("T", Type), ("ord", Function("ord", Any, Any, BoolSort())))
OrderMod = OrderMod.create()

BoolOrd = OrderMod.ORDMAKE(BoolSort(), Lambda([x,y], x == y))
def wf(mod):
    typeof(mod.ord) 


SyntaxError: unmatched ')' (1249398328.py, line 1)

# Sanity and Countermodels
Sanity:
axiom creation facilities should have ways of random testing. basic sanity



Nitpick

strip off out forall
upon failure of lemma do:
- search
- stronger nitpick. (tactics that mayb produce models that don't actually work?)
- 


## Manual FOL axioms
I can't get 
maybe backup axioms or vampire

## Hypothesis
https://hypothesis.readthedocs.io/en/latest/numpy.html

# MultiSovler Sledge



cvc5 has suport for trasncendtals

How could I support multiple solvers?
1. export from z3 ast.


Calc(lemma=cvc5_lemma)


Lemma()

nitpick counterexample generation.

dreal
https://cvc5.github.io/docs/cvc5-1.0.2/theories/transcendentals.html



In [ ]:
# https://courses.grainger.illinois.edu/cs576/sp2015/doc/isar-overview.pdf
# https://isabelle.in.tum.de/doc/isar-ref.pdf
class Lemma: # Isar?
    def __init__(self, goal):
        self.assume = []
        self.vars = []
        self.lemmas = []
        self.goal = goal
    def fix(self, var):
        self.vars.append(var)
    def assume(self, thm):
        self.assume.append(thm)
    def ornament(self, thm):
        return ForAll(vars, Implies(And(self.assume), thm))
    def have(self, thm, by=[]):
        self.lemmas.append(kd.lemma(self.ornament(thm), by=by+self.lemmas))
        return self
    def qed(self):
        return kd.lemma(self.goal, by=self.lemmas)


def sledge(thm, by=[]):
    # traverse theorem, get definitions recursively.
    # get helpful lemmas per theorem
    # call z3, call cvc5, call vampire
    # reduce output to needed bits.
    # find them in global theorem db

In [1]:
import z3
import knuckledragger as kd
class BaseSolver():
    def __init__(self):
        self.adds = []
        self.assert_tracks = []
        self.options = {}
    def add(self, thm):
        self.adds.append(thm)
    def assert_and_track(self, thm, name):
        self.assert_tracks.append((thm, name))
    def check(self):
        raise NotImplementedError
    def unsat_core(self):
        raise NotImplementedError
    def set(self, option, value):
        self.options[option] = value


import tempfile
import subprocess
class VampireSolver(BaseSolver):
    predefined = [
        z3.Function("abs", z3.RealSort(), z3.RealSort()),
    ]
    def __init__(self):
        super().__init__()
    def check(self):
        with open("/tmp/vampire.smt2", "w") as fp: #tempfile.NamedTemporaryFile() 
            fp.write((smtlib_datatypes([kd.theories.Interval.Interval])))
            fp.write(";;declarations\n")
            for f in kd.kernel.defns.keys():
                assert isinstance(f, z3.FuncDeclRef)
                if f not in self.predefined:
                    fp.write(f.sexpr())
                    fp.write("\n")
            fp.write(";;axioms\n")
            for e in self.adds:
                fp.write("(assert " + e.sexpr() + ")\n")
            for (thm,name) in self.assert_tracks:
                fp.write("(assert (! " + thm.sexpr() + " :named " + name + "))\n")
            fp.write("(check-sat)\n")
            fp.flush()
            #print(fp.readlines())
            cmd = ["vampire", fp.name, "--input_syntax", 
                   "smtlib2", #"--ignore_unrecognized_logic", "on",
                   "-t", str(self.options["timeout"] // 100) +"d", ]
            if len(self.assert_tracks) > 0:
                cmd.extend(["--output_mode", "ucore"])
            else:
                cmd.extend(["--output_mode", "smtcomp"])
            
            self.res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        res = self.res.stdout
        if b"unsat\n" in res or b"% SZS status Unsatisfiable" in res:
            self.status = z3.unsat
            return z3.unsat
        elif res == "sat":
            return z3.sat
        else:
            return z3.unknown
    def unsat_core(self):
        assert self.status == z3.unsat
        cores = self.res.stdout.decode().replace("unsat\n", "").replace("(", "").replace(")", "")
        cores = z3.Bools(cores)
        return cores


s = VampireSolver()
s.add(z3.RealVal(1) != z3.RealVal(1))
s.assert_and_track(z3.RealVal(1) != z3.RealVal(1), "knuckle_goal")
s.set("timeout", 1000)
s.check()
s.unsat_core()[0]

NameError: name 'smtlib_datatypes' is not defined

In [130]:
def smtlib_datatypes(dts : list[DatatypeSortRef]) -> str:
    s = Solver()
    for dt in dts:
        x = Const('x', dt)
        s.add(Exists([x], x == x)) # trivial constraint
    return s.to_smt2().replace("(check-sat)","").replace("""; benchmark generated from python API\n(set-info :status unknown)\n""","")

foo = DeclareSort("foo")
print(smtlib_datatypes([foo, kd.theories.Interval.Interval]))

(declare-sort foo 0)
 (declare-datatypes ((Interval 0)) (((mk (lo Real) (hi Real)))))
 (assert
 (exists ((x foo) )(= x x))
 )
(assert
 (exists ((x Interval) )(= x x))
 )




In [116]:
"""
    d.name()
    for i in range(d.num_constructors()):
        cons = d.constructor(i)
        for j in range(cons.arity()):
            acc = cons.accessor(i,j)
"""

dir(kd.theories.Interval.Interval.constructor(0))
kd.theories.Interval.Interval.constructor(0)

In [127]:
from z3 import *

# Create a new Z3 context
ctx = Context()

# Define the recursive type
MyType = Datatype('MyType')

# Add fields to the datatype
MyType.declare('mk_mytype', ('children', ArraySort(IntSort(), IntSort())))
MyType = MyType.create()

# Create a Z3 solver
solver = Solver()

# Add an instance of the datatype to the solver
x = Const('x', MyType)
solver.add(Exists([x], x == x))

# Print the SMT-LIB declaration of the datatype
#print(MyType)

# Print the SMT-LIB representation of the solver's assertions
print(solver.to_smt2().replace("(check-sat)","").replace("""; benchmark generated from python API\n(set-info :status unknown)\n""",""))
def print(datatype):


(declare-datatypes ((MyType 0)) (((mk_mytype (children (Array Int Int))))))
 (assert
 (exists ((x MyType) )(= x x))
 )




In [ ]:
; benchmark generated from python API
(set-info :status unknown)

## Better Triggers
Maybe a lighter weight thing to do is push a little extra bullshit into z3 to overcome it's barriers.

Lemma injection
Trigger injection

knuckeldummies. trigger terms
https://smt-workshop.cs.uiowa.edu/2023/slides/paper_11.pdf complkete trigger selection in satasifiability modulo theories https://ceur-ws.org/Vol-3429/paper11.pdf
https://dl.acm.org/doi/full/10.1145/3571748 Identifying Overly Restrictive Matching Patterns in SMT-based Program Verifiers (Extended Version)
https://mmoskal.github.io/smt/e-matching.pdf  E-matching for Fun and Prof

This is like what cody was talking about
interesting_terms =[]
dummy = FreshConst(  , prefix="knuckledummy)
for itemr in interesting_terms:
    s.add(Exists([dummy], dummy == iterm))

I should see if this is sufficient.


# Extraction Lambidfy
A good point is that we may want to extract python code, C code, numpy, pytorch, etc.

Codegen a string and eval it?


lambdafiy
https://docs.sympy.org/latest/modules/utilities/lambdify.html


numpy
pandas/polars

smt theory of dataframes.
https://arxiv.org/abs/2405.03057

I feel the need to havbe all kinds of trasnaltors. Register sine function as going to this and that. dreal, arb, calcium.

Lambdify is so hacky, I don't really trust it.
Shims for names.


In [ ]:
import sympy as sp
import knuckledragger.theories.Real as R

sp.lamdbify([x], sp.exp(sp.exp(x)))


In [78]:
import sympy
_lambdify_generated_counter = 0
def lambdify(xs, body):
    xs = (xs,) if isinstance(x, z3.ExprRef) else xs
    xs = map(str,xs)
    funcname = '_knuckle_lambdifygenerated'
    funcstr = f"""
def {funcname}({','.join(xs)}):
    return {body}
    """
    print(funcstr)
    global _lambdify_generated_counter
    filename = '<lambdifygenerated-%s>' % _lambdify_generated_counter
    _lambdify_generated_counter += 1
    c = compile(funcstr, filename, 'exec')
    namespace = {"sympy": sympy}
    funclocals = {}
    exec(c, namespace, funclocals)
    return funclocals[funcname]

from z3 import *
x = Real("x")
x1 = sp.symbols("x1")
lambdify(x, x**2 + 2*x + 1)(x1).factor()



def _knuckle_lambdifygenerated(x):
    return x**2 + 2*x + 1
    


(x1 + 1)**2

In [ ]:

def lambdify_defns():
    code = []
    for defn in kernel.defns:
        code.append(f"""
                    
                    defns.args"""


In [50]:
from sympy import lambdify
import sympy.abc
import knuckledragger.theories.Real as R
import knuckledragger as kd
import sympy as sp
import knuckledragger.smt as smt

x = sp.symbols("x")
f = sp.Lambda(x, sp.exp(sp.exp(x)))
lambdify([sp.abc.x], f, [smt, R])(smt.RealVal(1))

Lambda(Real, exp(exp(Real)))

In [61]:
def factor(t: smt.ExprRef) -> smt.ExprRef:
    expr = smt.FreshConst(t.sort(), prefix="knuckle_goal")
    G = smt.Goal()
    #for v in kd.kernel.defns.values():
    #    G.add(v.ax.thm)
    G.add(t == 0)
    G2 = smt.Tactic("factor").apply(G)[0]
    print(G2)
    return G2[len(G2) - 1].children()[0]

x, y = smt.Reals("x y")
factor(x**2 + 2*x + 1)

[x + 1 == 0]


x + 1

In [96]:
import sympy as sp
def mvt(t, x, a, b):
    eps = sp.symbols("\epsilon")
    dt = t.subs(x,eps).diff(eps)
    sol = sp.solve(sp.Eq(t.subs(x,a), t.subs(x,b) + dt * (b - a)), eps)
    return dt.subs(eps,sol[0])
x,a,b = sp.symbols("x a b")

mvt(x**2, x, a, b)*(b-a)

sp.singularities(mvt(sp.exp(x), x, a, b), (b-a))

    

{0}

In [ ]:
def singularity_removal(t):
    


In [62]:
x = sp.symbols("x")
sp.factor(x**2 + 3*x + 1)

x**2 + 3*x + 1

In [ ]:
ExprRef.to_sympy
def to_sympy(t : smt.ExprRef):
    

def from_sympy(x, t):
    return sp.lambdify([x], t, [kd.smt, R])(x)

factor, trigsimp, expand_trig, hyperexpand, 

diff



# Cond
Pattern matching
Chaining via If sucks,

Ben suggests pandas interfave as preexisting


In [19]:
import z3
# but still that class form might format better under ruff
def cond(*cases, default=None):
    sort = cases[0][1].sort()
    if default is None:
        s = z3.Solver()
        s.add(z3.Not(z3.Or([c for c, t in cases])))
        res = s.check()
        if res == z3.sat:
            raise Exception("Cases not exhaustive. Fix or give default", s.model())
        elif res != z3.unsat:
            raise Exception("Solver error. Give default", res)
        else:
            default = z3.FreshConst(sort,prefix="unreachable")
    acc = default
    for (c, t) in reversed(cases):
        if t.sort() != sort:
            raise Exception("Sort mismatch in cond", t, sort)
        acc = z3.If(c, t, acc)
    return acc

# Calc also has benefit of throwing error at right line, picking le lt, simp. etc.
# but maybe calc is still good for one liners?
# Calc can be chained as @=?
"""
def calc(vs, *cases):
    for v in vs:
        if v in cases:
            return cases[v]
    return None
"""
x = z3.Real("x")
cond((x > 0,  3*x),
     (x < 0,  2*x),
     (x == 0, 5*x))
cond((x < 0, 2*x),
     (x > 0,  3*x))


Exception: ('Cases not exhaustive. Fix or give default', [x = 0])

In [ ]:
class Cond:
    """
    Cond is a useful way to build up giant if-then-else expressions.
    """

    def __init__(self):
        self.clauses = []
        self.cur_case = None
        self.other = None
        self.sort = None
        self.default = None

    def when(self, c: smt.BoolRef) -> "Cond":
        assert self.cur_case is None
        assert isinstance(c, smt.BoolRef)
        self.cur_case = c
        return self

    def then(self, e: smt.ExprRef) -> "Cond":
        assert self.cur_case is not None
        if self.sort is not None:
            assert e.sort() == self.sort
        else:
            self.sort = e.sort()
        self.clauses.append((self.cur_case, e))
        self.cur_case = None
        return self

    def otherwise(self, e: smt.ExprRef) -> smt.ExprRef:
        assert self.default is None
        assert self.sort == e.sort()
        self.default = e
        return self.expr()

    def expr(self) -> smt.ExprRef:
        assert self.default is not None
        assert self.cur_case is None
        acc = self.default
        for c, e in reversed(self.clauses):
            acc = smt.If(c, e, acc)
        return acc

In [11]:
import knuckledragger as kd
Calc.__iand__ = Calc.eq

x = Real("x")
c = Calc([x], x) 
c &= x + 0 &= x + 0 + 0
# ah no such luck. Oh well. It's for the best probably. Too cute by half.


SyntaxError: invalid syntax (2554044391.py, line 6)

# z3 Parameters


https://microsoft.github.io/z3guide/programming/Parameters/



In [27]:
from z3 import *
cos = Function("cos", RealSort(), RealSort())
sin = Function("sin", RealSort(), RealSort())
x, y = Reals("x y")
sin_sum = ForAll([x,y], sin(x+y) == sin(x) * cos(y) + cos(x) * sin(y), qid="sin_sum")
l = sin(x + x) == 2 * sin(x) * cos(x)
s = Solver()
s.set("timeout", 1000)
s.set("instantiations2console", True)
#s.set("qi.max_multi_patterns", 3)
#s.set("lemmas2console", True)
#s.set("arith.solver", 5)
s.add(sin_sum)
s.add(Not(l))
s.check()

unsat

# Termination checking

Surely I have notes somewhere else?

Lex ordering of integers shouldn't be that bad
Could even use
V1 Nat
V2 Nat
v3 Nat

And V10 being well ordered ought the prove V1-9 are

Any algorithm running in less that N^10 ought to be provably terminating?



In [ ]:
V2 = kd.Record("V2", ("x", Nat), ("y", Nat))



# serialization

metamath import export?


In [214]:
from cvc5.pythonic import *

foo = Const("foo", ArraySort(IntSort(), BoolSort()))
x = Bool("x")
y = Int("y")
l = Lambda([y], BoolVal(True))
print(l.sort())
print(foo.sort())
foo == l
#foo == 


Bool
Array(Int, Bool)


SMTException: sort mismatch

In [ ]:
def proof_to_json(pfs: list[kd.kernel.Proof]):
    hashcons = {}
    pf_array = pfs
    i = 0
    while len(pfs) > i
        p = pf_array[i]
        len(pf_array)

        

def json_to_proof(j):
    for i in j:
        

# Database
Lemma search is super important. Also docs

Maybe I should make some kind of auto scraper.

Use AI to generate summaries

RAG


Smart lemma should return some guesses on failure of what may have gone wrong:

- definitions missing
- possibly useful theorems. by pattern match, by fingerprint index, ai query.
- countermodel - nitpick

I can't search induction lemmas or other schema. That's bad.
or `apply` a schema.
Maybe a Schema registry system.

lemma(  ,  ,apply=, auto=True, simp=True)







In [7]:
#https://stackoverflow.com/questions/328851/printing-all-instances-of-a-class
import gc
import knuckledragger as kd
import knuckledragger.theories.Real as R
db = [obj for obj in gc.get_objects() if kd.kernel.is_proof(obj)]
db

def blast(thm):
    return kd.kernel.lemma(thm,by=[obj for obj in gc.get_objects() if kd.kernel.is_proof(obj)])


[|- ForAll([x, y], plus(x, y) == x + y),
 |- ForAll(x, plus(x, 0) == x),
 |- ForAll([x, y], plus(x, y) == plus(y, x)),
 |- ForAll([x, y, z],
        plus(x, plus(y, z)) == plus(plus(x, y), z)),
 |- ForAll([x, y], mul(x, y) == x*y),
 |- ForAll(x, mul(x, 0) == 0),
 |- ForAll(x, mul(x, 1) == x),
 |- ForAll([x, y], mul(x, y) == mul(y, x)),
 |- ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z)),
 |- ForAll([x, y, z],
        mul(x, plus(y, z)) == plus(mul(x, y), mul(x, z))),
 |- ForAll(x, abs(abs(x)) == abs(x)),
 |- ForAll(x, abs(-x) == abs(x)),
 |- ForAll(x, abs(x) >= 0),
 |- ForAll(x, nonneg(x) == (x >= 0)),
 |- ForAll([x, y], max(x, y) == max(y, x)),
 |- ForAll([x, y, z], max(x, max(y, z)) == max(max(x, y), z)),
 |- ForAll(x, max(x, x) == x),
 |- ForAll([x, y], max(x, y) >= x),
 |- ForAll([x, y], max(x, y) >= y),
 |- ForAll([x, y], min(x, y) == min(y, x)),
 |- ForAll([x, y, z], min(x, min(y, z)) == min(min(x, y), z)),
 |- ForAll(x, min(x, x) == x),
 |- ForAll([x, y], min(x, y) <= 

In [ ]:


def search_decl(f : z3.FuncDeclRef):
    yield from filter(lambda thm: f in map(ts.decl() thms)
def search_pat(pat : z3.ExprRef):
    yield from filter(lambda thm: pat in thm.children(), thms)
def search_text():
    pass




We shouldn't find anything with sorts that aren't in our expression (?) Uh, I dunno. Maybe we should


In [98]:
from collections import Counter
import sys
def subterm_set(x):
    s = set()
    todo = [x]
    for x in todo:
        if x in s:
            continue
        s.add(x)
        todo.extend(x.children())        
    return s
def subterms(x):
    todo = [x]
    for x in todo:
        yield x
        todo.extend(x.children())

def decls(t):
    return {f.decl() for f in subterm_set(t) if z3.is_app(f)}

def defns(t):
    return {kd.kernel.defns[decl].ax for decl in decls(t) if decl in kd.kernel.defns}

def feature_vector(thm):
    #sterms = subterms(thm)
    #decls = Counter(s.decl() for s in sterms) # count?
    decls = Counter(s.decl() for s in subterms(thm) if z3.is_app(s))
    #sorts = Counter(s.sort() for s in subterms(thm) if z3.is_app(s))
    #sys.getrefcount(thm) # or of proof?
    # has name? also useful feature
    #print(decls, sorts)
    return decls #+ sorts
    #words = Counter(s.decl().name() for s in sterms)
    # pretty print

# We should not
def dot(x : Counter,y : Counter) -> float:
    xnorm = sum(v**2 for v in x.values())**0.5
    ynorm = sum(v**2 for v in y.values())**0.5
    common_keys = x.keys() & y.keys()
    #return len(common_keys)
    return sum(x[k] * y[k] for k in common_keys) / xnorm / ynorm
        
import gc
import knuckledragger as kd
import knuckledragger.theories.Real as R
import knuckledragger.theories.Interval as I
import knuckledragger.utils
import inspect
def search(thm):
    fv = feature_vector(thm)
    print(fv)
    pfs = [(modname, modname + "." + thmname, thm) for modname, mod in kd.utils.lemma_db().items() for thmname, thm in mod.items()]
    return sorted([(dot(fv, feature_vector(obj.thm)) , thmname, obj) for modname, thmname, obj in pfs if kd.kernel.is_proof(obj)], key=lambda x: -x[0])
    #return sorted([(dot(fv, feature_vector(obj.thm)) , obj) for obj in gc.get_objects() if kd.kernel.is_proof(obj)], key=lambda x: -x[0])
def search_locals(thm):
    #print(locals())
    # Get the current call stack
    stack = inspect.stack()
    
    # The caller's frame is the second in the stack (index 1)
    caller_frame = stack[1].frame
    
    # Get the local variables of the caller's frame
    caller_locals = caller_frame.f_locals
    print([(name, x) for name, x in caller_locals.items() if kd.kernel.is_proof(x)]) 
    print(caller_locals.keys())
p,q,r = z3.Reals("p q r")
search(R.abs(x) + R.max(p,q) + R.max(r,q))
#defns(R.abs(x) + R.max(p,q) + R.max(r,q))
#decls(z3.ForAll([x], 0 <=  R.abs(x) + R.max(p,q) + R.max(r,q)))
foo = R.abs_ge_0
search_locals(R.abs(x) + R.max(p,q) + R.max(r,q))

Counter({+: 2, max: 2, q: 2, abs: 1, r: 1, x: 1, p: 1})
[('_', |- ForAll(x, abs(x) >= 0)), ('foo', |- ForAll(x, abs(x) >= 0)), ('_89', |- ForAll(x, abs(x) >= 0))]
dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', 'gc', 'db', 'obj', '_i2', 'kd', '_2', '_i3', '_3', '_i4', '_i5', '_5', '_i6', 'R', '_6', '_i7', '_7', '_i8', '_i9', '_i10', '_i11', '_i12', 'cond', '_12', '_i13', '_i14', '_i15', 'z3', 'x', '_i16', '_i17', '_i18', '_18', '_i19', '_i20', '_i21', '_i22', 'Counter', 'subterms', 'feature_vector', 'dot', 'search', '_i23', '_i24', 'y', 'z', '_i25', 'sys', '_i26', '_i27', '_i28', '_i29', '_i30', '_i31', '_i32', '_32', '_i33', '_33', '_i34', '_34', '_i35', '_i36', '_36', '_i37', '_37', '_i38', '_38', '_i39', '_i40', '_40', '_i41', '_i42', '_42', '_i43', '_43', '_i44', '_44', '_i45', '_45', '

In [94]:
foo = R.abs_ge_0

In [6]:
from kdrag.utils import prompt
from openai import OpenAI
import os
def openai_chat(q):
    client = OpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt(q),
            }
        ],
        model="gpt-4o-mini",
    )
    return chat_completion.choices[0].message.content

res = openai_chat("What would be some natural theories or theorems to add next? Please give code")
print(res)
# it's slow because it's not streaming

Here are a few ideas for natural theories or theorems you could add to the Knuckledragger project. Each of these comes with Python code that builds upon existing structures and lays out axioms, definitions, and lemmas.

### 1. Basic Properties of Even and Odd Numbers

This theory will provide definitions for even and odd integers, along with some basic properties.

```python
# --- /home/philip/Documents/python/knuckledragger/kdrag/theories/even_odd.py

import kdrag as kd
import kdrag.smt as smt

Z = smt.IntSort()

# Define the even and odd predicates
even = smt.Function("even", Z, smt.BoolSort())
odd = smt.Function("odd", Z, smt.BoolSort())

# A number is even if it can be expressed as 2k for some integer k
kd.axiom(smt.ForAll([n], even(n) == smt.Exists([k], n == 2 * k)))

# A number is odd if it can be expressed as 2k + 1 for some integer k
kd.axiom(smt.ForAll([n], odd(n) == smt.Exists([k], n == 2 * k + 1)))

# Basic properties
even_zero = kd.lemma(smt.ForAll([], even(0)), by=[])
odd_

In [5]:
print(res.choices[0].message.content)

Given the wide array of existing theories and theorems in the Knuckledragger project, a natural next step could be to explore extending it with additional mathematical and logical concepts. Here are some suggestions for new theories or theorems, along with sample code to implement them:

---

### 1. Group Theory

Group theory is foundational to many areas of mathematics. Here, we define a simple group and prove some properties.

```python
import kdrag as kd
import kdrag.smt as smt

# Defining the Group structure
Group = kd.notation.Record("Group", ("G", smt.SetSort(smt.IntSort())), ("op", smt.FunctionSort(smt.IntSort(), smt.IntSort(), smt.IntSort())))
g, h, k = smt.Ints("g h k")
G = smt.Const("G", Group)

# Defining the group operation
op = G.op

# Defining group axioms
assoc = kd.axiom(smt.ForAll([g, h, k], op(op(g, h), k) == op(g, op(h, k))))
identity = smt.Const("identity", smt.IntSort())
identity_axiom = kd.axiom(smt.Exists([identity], smt.ForAll([g], smt.And(op(identity, g) == g, 

In [22]:
import knuckledragger.theories.Real as Real
import knuckledragger.theories.Complex as Complex
import knuckledragger.theories.Interval as Interval
import knuckledragger.theories.Int as Int
import knuckledragger.theories.Nat as Nat
import knuckledragger as kd
from openai import OpenAI
def ai(thm):
    mods = [
    Real,
    Complex,
    Interval,
    Int,
    Nat
    ]
    kd.theories.__dict__
    prompt = ["""
    This is the source of a library called knuckledragger, a python proof assistant. It uses Z3.
    """]
    for mod in mods:
        with open(mod.__file__) as f:
            prompt.append("##### " + mod.__name__ + "\n")
            prompt.append(f.read())
        
    prompt.append("I'm trying to prove the following lemma:")
    prompt.append(str(thm))
    prompt.append("Do you have suggestions for what I should write. Please be as succinct as possible. Only show code. You may define lemmas if you need to.")
    #prompt.append("what do you think I should write next?")
    #prompt.append("Write more stuff to go in the Complex module.")

    client = OpenAI(
        # This is the default and can be omitted
        api_key=os.environ.get("OPENAI_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "\n".join(prompt),
            }
        ],
        model="gpt-4o",
    )
    return chat_completion

res = ai(ForAll([]))
res


ChatCompletion(id='chatcmpl-9t5gIlah6Bm0WcEdCVYPYTmCPACLA', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="To build a foundational module for dealing with complex numbers in your `knuckledragger` proof assistant, you can leverage the properties of real numbers and extend them accordingly. A comprehensive `Complex` module can incorporate definitions and properties of complex arithmetic, similar to what you have in the `Real` module.\n\nHere's an initial implementation:\n\n```python\n##### knuckledragger.theories.Complex\n\nimport z3\nfrom z3 import ForAll, Function, And, Not\nimport knuckledragger as kd\n\n# Define the sort for complex numbers\nComplexSort = z3.Datatype('Complex')\nComplexSort.declare('mk_complex', ('real', z3.RealSort()), ('imag', z3.RealSort()))\nComplexSort = ComplexSort.create()\n\nC = ComplexSort\nR = z3.RealSort()\nx, y, z = z3.Reals('x y z')\na, b, c = z3.Consts('a b c', C)\n\n# Define the operations for compl

In [30]:
__name__

# Access the current module
import inspect
def auto_register():
    stack = inspect.stack()
    caller_frame = stack[1]
    print("Caller function:", caller_frame)
    print(dir(caller_frame[0]))
    print(caller_frame[1])
    print(caller_frame[2])
    caller_module = inspect.getmodule(caller_frame[0])
    if caller_module:
        print("Caller module:", caller_module.__name__)
    else:
        print("Caller module not found")

auto_register()

Caller function: FrameInfo(frame=<frame at 0x57ce1b99a8e0, file '/tmp/ipykernel_3779874/2569732049.py', line 18, code <module>>, filename='/tmp/ipykernel_3779874/2569732049.py', lineno=18, function='<module>', code_context=['auto_register()\n'], index=0)
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'f_back', 'f_builtins', 'f_code', 'f_globals', 'f_lasti', 'f_lineno', 'f_locals', 'f_trace', 'f_trace_lines', 'f_trace_opcodes']
/tmp/ipykernel_3779874/2569732049.py
18
Caller module not found


In [54]:
import knuckledragger.theories.Real as Real
from knuckledragger.kernel import is_proof
for n, p in Real.__dict__.items():
    if is_proof(p):
       display(n, HTML(p._repr_html_()))

dir(Real.plus_def.thm)


'plus_def'

'plus_0'

'plus_comm'

'plus_assoc'

'mul_def'

...

'mul_comm'

'mul_assoc'

'mul_distrib'

'abs_def'

['__add__',
 '__and__',
 '__bool__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '__xor__',
 '_repr_html_',
 'arg',
 'as_ast',
 'ast',
 'body',
 'children',
 'ctx',
 'ctx_ref',
 'decl',
 'eq',
 'from_string',
 'get_id',
 'hash',
 'is_exists',
 'is_forall',
 'is_lambda',
 'no_pattern',
 'num_args',
 'num_no_patterns',
 'num_patterns',
 'num_vars',
 'params',
 'pattern',
 'qid',
 'serialize',
 'sexpr',
 'skolem_id',
 'sort',
 'sort_kind',
 'translate',
 'use_pp',
 'var_name',
 'var_sort',
 'weight']

In [43]:
Real.plus_def.thm.use_pp()
repr(Real.plus_def.thm)
Real.plus_def._repr_html_()

'&#8870;&forall;x, y : plus(x, y) = x + y'

In [8]:
import sys
import knuckledragger as kd
import knuckledragger.theories.Real as Real
lemma_db = {"defns": kd.kernel.defns}

def build_lemma_db():
    for modname,mod in sys.modules.items():
        thms = {name : thm for name,thm in mod.__dict__.items() if kd.kernel.is_proof(thm)}
        if len(thms) > 0:
            lemma_db[modname] = thms
build_lemma_db()
lemma_db



AttributeError: module 'knuckledragger.kernel' has no attribute 'defns'

Caching might be nice
Could blacklist dumb modules

In [10]:
import sys
import knuckledragger as kd
import knuckledragger.theories.Real as Real
def lemma_db():
    db = {}
    for modname,mod in sys.modules.items():
        thms = {name : thm for name,thm in mod.__dict__.items() if kd.kernel.is_proof(thm)}
        if len(thms) > 0:
            db[modname] = thms
    return db
lemma_db()

{'knuckledragger.theories.Real': {'plus_def': |- ForAll([x, y], plus(x, y) == x + y),
  'plus_0': |- ForAll(x, plus(x, 0) == x),
  'plus_comm': |- ForAll([x, y], plus(x, y) == plus(y, x)),
  'plus_assoc': |- ForAll([x, y, z],
         plus(x, plus(y, z)) == plus(plus(x, y), z)),
  'mul_def': |- ForAll([x, y], mul(x, y) == x*y),
  'mul_zero': |- ForAll(x, mul(x, 0) == 0),
  'mul_1': |- ForAll(x, mul(x, 1) == x),
  'mul_comm': |- ForAll([x, y], mul(x, y) == mul(y, x)),
  'mul_assoc': |- ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z)),
  'mul_distrib': |- ForAll([x, y, z],
         mul(x, plus(y, z)) == plus(mul(x, y), mul(x, z))),
  'abs_def': |- ForAll(x, abs(x) == If(x >= 0, x, -x))}}

In [88]:
import sys
lemma_db = {}
for modname,mod in sys.modules.items():
    if "knuckledragger" in modname:
        print(modname)
        for thmname, proof in mod.__dict__.items():
            if is_proof(proof):
                display(thmname, HTML(proof._repr_html_()))

__editable___knuckledragger_0_1_1_finder
knuckledragger.kernel
knuckledragger
knuckledragger.theories
knuckledragger.theories.Real


'plus_def'

'plus_0'

'plus_comm'

'plus_assoc'

...

'mul_1'

'mul_comm'

'mul_assoc'

'mul_distrib'

'abs_def'

# Simp
Use eggmt 

We can use match_ to do a simp routine, knuckledragger tactic.

Idea: use eggllg
https://egglog-python.readthedocs.io/latest/reference/bindings.html

https://fadoss.github.io/maude-bindings/

to prolog?

Old simpler pmatches

In [ ]:
def pmatch_db(
    t: smt.ExprRef, pat: smt.ExprRef
) -> Optional[dict[smt.ExprRef, smt.ExprRef]]:
    """
    Pattern match t against pat. Variables are constructed as `smt.Var(i, sort)`.
    Returns substitution dict if match succeeds.
    Returns None if match fails.
    Outer quantifier (Exists, ForAll, Lambda) in pat is ignored.
    """
    if smt.is_quantifier(pat):
        pat = pat.body()
    subst = {}
    todo = [(t, pat)]
    while len(todo) > 0:
        t, pat = todo.pop()
        if t.eq(pat):
            continue
        if smt.is_var(pat):
            if pat in subst:
                if not subst[pat].eq(t):
                    return None
            else:
                subst[pat] = t
        elif smt.is_app(t) and smt.is_app(pat):
            if pat.decl() == t.decl():
                todo.extend(zip(t.children(), pat.children()))
            else:
                return None
        else:
            raise Exception("Unexpected subterm or subpattern", t, pat)
    return subst


def pmatch(
    t: smt.ExprRef, vs: list[smt.ExprRef], pat: smt.ExprRef
) -> Optional[dict[smt.ExprRef, smt.ExprRef]]:
    """Pattern match t against pat considering vs as variables. Returns substitution dictionary if succeeds"""
    subst = {}
    todo = [(t, pat)]
    while len(todo) > 0:
        t, pat = todo.pop()
        if t.eq(pat):
            continue
        if any(pat.eq(v) for v in vs):
            if pat in subst:
                if not subst[pat].eq(t):
                    return None
            else:
                subst[pat] = t
        elif smt.is_app(pat):
            if smt.is_app(t) and pat.decl() == t.decl():
                todo.extend(zip(t.children(), pat.children()))
            else:
                return None
        else:
            raise Exception("Unexpected subterm or subpattern in pmatch", t, pat)
    return subst

In [29]:
# hmm. There's an interesting z3 tactic
# https://microsoft.github.io/z3guide/docs/strategies/summary/#tactic-demodulator
# https://github.com/draperlaboratory/cozy/blob/main/cozy/claripy_ext.py#L10
from z3 import *
g  = Goal()
f = Function("f", IntSort(), IntSort())
x= Int("x")
z = FreshInt("z")
g.add(ForAll([x], f(f(x)) == x))
g.add(z == f(f(x)))
g2 = Tactic("demodulator").apply(g)
len(g2[0])
g2[0][1]

def knuck_tact(t, name, **kwargs):
    g = Goal()
    g.add(t)
    res = Tactic(name, **kwargs).apply(g)
    t1 = res[0]
    l = kd.lemma(t == res[0])# kd.lemma Implies(t, res[0]), Implies(res[0], t)
    t1.pf = l
    return t1

knuck_tact(t, "demodulator")
knuck_tact(t, "macro-finder")
knuck_tact(t, "simplify")
# ctx-simplify euf-completion
# solve-eqs

# elim-rpedicates subsumes



z!17 == x

In [22]:
import knuckledragger as kd
import knuckledragger.theories.Real as R
from z3 import *
G = Goal()
for k,v in kd.kernel.defns.items():
    G.add(v.ax.thm)

myexpr = Function("myexpr", RealSort(), BoolSort())
G.add(myexpr(R.max(3,4)))
G.add(x == R.max(3,4))
print(G)
for tac in ["simplify", "demodulator", "ctx-solver-simplify", "propagate-values", "macro-finder", "solve-eqs", "ctx-simplify", "elim-predicates"]:
    G2 = Tactic(tac).apply(G)
    print(tac, G2)





[ForAll(x, abs(x) == If(x >= 0, x, -x)),
 ForAll(x, nonneg(x) == (abs(x) == x)),
 ForAll([x, y], max(x, y) == If(x >= y, x, y)),
 myexpr(max(3, 4)),
 x == max(3, 4)]
simplify [[ForAll(x, abs(x) == If(x >= 0, x, -1*x)),
  ForAll(x, nonneg(x) == (abs(x) == x)),
  ForAll([x, y], max(x, y) == If(x >= y, x, y)),
  myexpr(max(3, 4)),
  x == max(3, 4)]]
demodulator [[ForAll(x, abs(x) == If(x >= 0, x, -x)),
  ForAll([x, y], max(x, y) == If(x >= y, x, y)),
  myexpr(If(3 >= 4, 3, 4)),
  x == If(3 >= 4, 3, 4)]]
ctx-solver-simplify [[ForAll(x, abs(x) == If(x >= 0, x, -x)),
  ForAll(x, nonneg(x) == (abs(x) == x)),
  ForAll([x, y], max(x, y) == If(x >= y, x, y)),
  myexpr(max(3, 4)),
  x == max(3, 4)]]
propagate-values [[ForAll(x, abs(x) == If(x >= 0, x, -x)),
  ForAll(x, nonneg(x) == (abs(x) == x)),
  ForAll([x, y], max(x, y) == If(x >= y, x, y)),
  myexpr(max(3, 4)),
  x == max(3, 4)]]
macro-finder [[myexpr(4), x == 4]]
solve-eqs [[ForAll(x, abs(x) == If(x >= 0, x, -1*x)),
  ForAll(x, nonneg(x) ==

AttributeError: 'str' object has no attribute 'ctx'

In [54]:
def simp(t):
    expr = FreshConst(t.sort(), prefix="knuckle_goal")
    G = Goal()
    for v in kd.kernel.defns.values():
        G.add(v.ax.thm)
    G.add(expr == t)
    G2 = Tactic("elim-predicates").apply(G)[0]
    return G2[len(G2)-1].children()[1]

simp(R.max(8, R.max(,4)))

If(8 >= If(x >= 4, x, 4), 8, If(x >= 4, x, 4))

In [1]:
import maude
maude.init()
m = maude.getModule('NAT')
t = m.parseTerm('2 * 3')
t.reduce()
print(t)

6


In [6]:
sy = t.symbol()
str(sy)

's_'

In [ ]:
def maude_sort_to_z3_sort()

def maude_to_z3(t) -> ExprRef:
    f = t.symbol()
    Function(str(f), *[f.f.domainKind(i) for i in range(f.arity())], f.rangeSort())
    maude_to_z3(arg) for arg in t.arguments()

def z3_to_maude(t) -> maude.Term:
    # very similar to tptp

In [1]:
from kdrag.all import *

def maude_of_expr(e):
    fp.write("fmod KNUCKLE is\n")
    for decl in decls:
        fp.write(f"op {decl.name()} : {" -> "join(decl.domain(i) for i in range(decl.arity())} -> {decl.range()} .")
    for t in thms:
        body, vs = open_binders(t)
        for v in vs:
            fp.write(f"var {v} : {v.sort()} .")
        fp.write(f"eq {body} .")
    fp.write("endfm")




Admitting lemma ForAll([x, y, z], mul(x, mul(y, z)) == mul(mul(x, y), z))
Admitting lemma ForAll(x, Implies(x >= 0, sqrt(x) >= 0))
Admitting lemma ForAll(x, Implies(x >= 0, sqrt(x)**2 == x))
Admitting lemma ForAll(x, Implies(x >= 0, sqrt(sqr(x)) == x))


In [ ]:
Nat = DeclareSort("NAT")
zero = Function("zero", Nat)
succ = Function("succ", Nat, Nat)
add = Function("add", Nat, Nat, Nat)
x,y,z = Consts("x y z", Nat)
ForAll([x], add(zero(), x) == x))

https://fadoss.github.io/maude-bindings/

In [21]:
c = 0

In [46]:
import maude
c += 1 
code = f"""
fmod KNUCKLE{c} is
sort Nat .
op zero : -> Nat .
op s_ : Nat -> Nat .
op _+_ : Nat Nat -> Nat .
vars N M : Nat .
eq zero + N = N .
eq s N + M = s (N + M) .
endfm\n"""
maude.init()
maude.input(code)
#maude.input("red zero + (s zero + s zero) .\n")
#maude.input("show sorts KNUCKLE4 .")
#maude.getModules()
m = maude.getModule(f'KNUCKLE{c}')
t = m.parseTerm("zero + (s zero + s zero)")
t.reduce()
t.arguments()


<maude.ArgumentIterator; proxy of <Swig Object of type 'EasyArgumentIterator *' at 0x7066e0536d90> >

In [55]:
import kdrag
from kdrag.all import *
x,y,z = kdrag.smt.Reals("x y z")
succ_0 = smt.ForAll([x], x + 0 == x)


def rule_of_theorem(thm):
    if smt.is_quantifier(thm):
        if thm.is_forall():
            vs, body = kdrag.utils.open_binder(thm)
    else:
        vs = []
    print(body)
    if not smt.is_eq(body):
        raise Exception("Not an equation")
    lhs, rhs = body.children()
    return vs ,lhs, rhs
vs, lhs, rhs = rule_of_theorem(succ_0)
#assert succ_0.eq(smt.ForAll(vs, lhs == rhs))

X + 0 == X


AssertionError: 

In [63]:
from typing import Optional
def pmatch(
    t: smt.ExprRef, vs : list[smt.ExprRef], pat: smt.ExprRef
) -> Optional[dict[smt.ExprRef, smt.ExprRef]]:
    subst = {}
    todo = [(t, pat)]
    while len(todo) > 0:
        t, pat = todo.pop()
        if t.eq(pat):
            continue
        if pat in vs:
            if pat in subst:
                if not subst[pat].eq(t):
                    return None
            else:
                subst[pat] = t
        elif smt.is_app(t) and smt.is_app(pat):
            if pat.decl() == t.decl():
                todo.extend(zip(t.children(), pat.children()))
            else:
                return None
        else:
            raise Exception("Unexpected subterm or subpattern in pmatch", t, pat)
    return subst

def rewrite1(t, vs, lhs, rhs):
    subst = pmatch(t, vs, lhs)    
    if subst is not None:
        return smt.substitute(rhs, *subst.items())
    return None

vs, lhs, rhs = rule_of_theorem(succ_0)
rewrite1(y + 0, vs, lhs, rhs)
        

X + 0 == X


y

In [74]:
def rewrite_star(t, rules):
    if smt.is_app(t):
        t = t.decl()(*[rewrite(arg, rules) for arg in t.children()])
        # TODO: index by decl
        while True:
            done = True
            for vs, lhs, rhs in rules:
                res = rewrite1(t, vs, lhs, rhs)
                if res is not None:
                    print(t, "-->", res)
                    # proof.append(t,res, rulen)
                    t = res
                    done = False
            if done:
                return t
    else:
        return t

def rewrite(t, rules):
    if smt.is_app(t):
        t = t.decl()(*[rewrite(arg, rules) for arg in t.children()])
        for vs, lhs, rhs in rules:
            res = rewrite1(t, vs, lhs, rhs)
            if res is not None:
                return res
    return t

# a different evaluation pattern
def rewrite_star2(t,rules):
    while True:
        t2 = rewrite(t, rules)
        if t2.eq(t):
            return t2
        t = t2

t = (y + 0) + 0
rewrite(t, [rule_of_theorem(succ_0)])
rewrite_star(t, [rule_of_theorem(succ_0)])

def decl_index(rules):
    return {lhs.decl() : (vs, lhs, rhs) for vs, lhs, rhs in rules}


X + 0 == X


y

In [39]:
# rewrite exactly once
Rule = tuple[list[smt.ExprRef], smt.ExprRef, smt.ExprRef]
def rewrite(t, rules : dict[smt.FuncDeclRef, list[Rule]]):
    if smt.is_app(t):
        decl = t.decl()
        c = t.children()
        for i in range(len(c)):
            res = rewrite(c[i], rules)
            if res is not None:
                c[i] = res
                return decl(*c)
        for rule in rules[decl]:
            res = rewrite1(t,vs,lhs,rhs)
            if res is not None:
                return res
# or reorder loops to rewrte as close to root as possible.
def rewrite(t, rules : dict[smt.FuncDeclRef, list[Rule]]):
    if smt.is_app(t):
        decl = t.decl()
        for rule in rules[decl]:
            res = rewrite1(t,vs,lhs,rhs)
            if res is not None:
                return res
        c = t.children()
        for i in range(len(c)):
            res = rewrite(c[i], rules)
            if res is not None:
                c[i] = res
                return decl(*c)
# decl zippers
Zipper = tuple(FuncDeclRef, list[ExprRef], list[ExprRef])


True

In [ ]:
def build_index(rules):
    index = defaultdict(list)
    if not isinstance(rules, list):
        rules = [rules]
    for rule in rules:
        if isinstance(rule, Rule) or isinstance(rule, tuple):
            vs,lhs,rhs = rule
            index[lhs.decl()].append(rule)
        elif kdrag.is_proof(rule):
            vs,lhs,rhs = rule_of_theorem(rule.thm)
        elif isinstance(rule, smt.BoolRef):
            vs,lhs,rhs = rule_of_theorem(rule)
        else:
            raise Exception("Unexpected rule format", rule)
        index[lhs.decl()].append(Rule(vs,lhs,rhs))
    return index

def rewrite(t, rules):
    if not isinstance(rules,dict):
        rules = build_index(rules)
    return rewrite(t, rules) # just inline it here.
 

There are at least 3 versions of what we may want our rules to be.
vs,lhs,rhs
forall x, lhs == rhs
Proof(forall x, lhs == rhs)

In [17]:
maude.getModules()

(fmod BOOL,
 fmod TRUTH-VALUE,
 fmod BOOL-OPS,
 fmod TRUTH,
 fmod EXT-BOOL,
 fmod INITIAL-EQUALITY-PREDICATE,
 fmod NAT,
 fmod INT,
 fmod RAT,
 fmod FLOAT,
 fmod STRING,
 fmod CONVERSION,
 fmod RANDOM,
 fmod BOUND,
 fmod QID,
 fth TRIV,
 fth STRICT-WEAK-ORDER,
 fth STRICT-TOTAL-ORDER,
 fth TOTAL-PREORDER,
 fth TOTAL-ORDER,
 fth DEFAULT,
 fmod LIST,
 fmod WEAKLY-SORTABLE-LIST,
 fmod SORTABLE-LIST,
 fmod WEAKLY-SORTABLE-LIST',
 fmod SORTABLE-LIST',
 fmod SET,
 fmod LIST-AND-SET,
 fmod SORTABLE-LIST-AND-SET,
 fmod SORTABLE-LIST-AND-SET',
 fmod LIST*,
 fmod SET*,
 fmod MAP,
 fmod ARRAY,
 fmod NAT-LIST,
 fmod QID-LIST,
 fmod QID-SET,
 fmod META-TERM,
 fmod META-CONDITION,
 fmod META-STRATEGY,
 fmod META-MODULE,
 fmod META-VIEW,
 fmod META-LEVEL,
 fmod LEXICAL,
 mod COUNTER,
 mod LOOP-MODE,
 mod CONFIGURATION,
 fmod KNUCKLE,
 fmod KNUCKLE2,
 fmod KNUCKLE3,
 fmod KNUCKLE4)

In [98]:
from z3 import *

Math = DeclareSort("Math")
add = Function("add", Math, Math, Math)
dir(add(x,y) == x)
(add(x,y) == x).decl().name()

'='

In [121]:
import egglog.bindings as eggbnd
from z3 import *

Math = DeclareSort("Math")
add = Function("add", Math, Math, Math)
zero = Const("zero", Math)
one =  Const("one", Math)

sig = [Math, add, zero, one]

x,y = Consts("x y", Math)
add(zero,one) == one
rules =[
ForAll([x], add(x,zero) == x),
ForAll([x,y], add(x,y) == add(y,x)),
]

def z3_to_egglog(e, vars=[]):
    if e in vars:
        return e.sexpr()
    else:
        head, args = e.decl(), e.children()
        return f"({head.name()} {' '.join(z3_to_egglog(a, vars) for a in args)})"

from knuckledragger.utils import open_binder
def simp(e, rules, sig, n=10):
    output = []
    #output.append("(declare-sort TERM)")
    for f in sig:
        if isinstance(f, z3.FuncDeclRef):
            dom = " ".join([f.domain(i).name() for i in range(f.arity())])
            output.append(f"(function {f.name()}  ({dom}) {f.range().name()})")
        elif isinstance(f, z3.SortRef):
            output.append(f"(datatype {f.name()})")
        elif isinstance(f, z3.ExprRef):
            output.append(f"(function {f.sexpr()} () {f.sort().name()})")
    for r in rules:
        assert r.is_forall()
        vs, r = open_binder(r)
        assert r.decl().name() == "="
        lhs, rhs = r.children()
        output.append(f"(rewrite {z3_to_egglog(lhs,vars=vs)} {z3_to_egglog(rhs, vars=vs)})")
    output.append(f"(let knuck_e {z3_to_egglog(e)})")
    output.append(f"(run {n})")
    output.append("(extract knuck_e)")
    return output


egraph = eggbnd.EGraph()
for c in simp(add(zero,one), rules, sig):
    print(c)
    commands = egraph.parse_program(c)
    res = egraph.run_program(*commands)
print(res)


(datatype Math)
(function add  (Math Math) Math)
(function zero () Math)
(function one () Math)
(rewrite (add X (zero )) X)
(rewrite (add X Y) (add Y X))
(let knuck_e (add (zero ) (one )))
(run 10)
(extract knuck_e)
['(one)']


## By Definitions

delta expansion.
Also a simp routine doing this would be nice.


Should I make a db of everything that comes through?
Maybe some config parameters?

I could also make the Kernel entities objects instead of global.



It's too bad that we lose the nice type definition of the manual style.
fact = define(  , lambda fact: If  , fix=True)

Scan for symbol being in rhs
aprove? NaTT?

Maybe lemma should not do the clever checking and that should be in a util wrapper.



In [ ]:

def decls(e : ExprRef): #get_sig. get_sig might also collect the types?
    if is_var(e):
        return
    else:
        yield e.decl()
        yield from c in e.children() for d in decls(c)
def by_defn(thm: z3.BoolRef, by=[]) -> Proof:  # delta expansion
    lemma(thm, by=[kernel.defn[decls(thm)]] + by)
    

## Lambda Eval

I've complained a bit before that it's crazy python doesn't have a good lambda library.
Well, it kind of does.


Lambda eval.
Z3 does do capture avoinding substitution. It can take care of de bruijn indices.
vs z3 simplify

Locally nameless matching. FreshConst + substitute makes this easy

Then we can use regular python pattern matching for lambda matching. Pretty cool!
I don't understand how the property mechanism works.

https://chargueraud.org/research/2009/ln/main.pdf 

In [51]:
"""
z3.ExprRef.head = property(lambda self: self.decl().kind())
z3.ExprRef.args = property(lambda self: [self.arg(i) for i in range(self.num_args())])
z3.ExprRef.__match_args__ = ["head", "args"]
z3.QuantifierRef.open_term = property(lambda self: vars = FreshConst() (return vars, subst(self.body, []))) 
z3.QuantifierRef.__match_args__ = ["open_term"]

z3.QuantifierRef.__matmul__ = lambda self, other: z3.substitute(self.body, zip([z3.Var(n) for n in range(len(other)) , other]))
"""
def open_binder(l : QuantifierRef):
    vars = []
    nvars = l.num_vars()
    for i in range(nvars):
        sort = l.var_sort(i)
        vars.append((Var(nvars - i - 1, sort) , FreshConst(sort, prefix=l.var_name(i))))
    return [x for (_,x) in vars], substitute(l.body(), vars)



open_binder(Lambda([x, y, z], foo(x,bar(bar(y)))))
open_binder(ForAll([x, y, z], foo(x,bar(bar(y))) == bar(z)))

def instan(f : QuantifierRef, *x):
    nvars = f.num_vars()
    assert len(x) == nvars
    vars = []
    for i in range(nvars):
        sort = f.var_sort(i)
        vars.append((Var(nvars - i - 1, sort), x[i]))
    return substitute(f.body(), *vars)

instan(ForAll([x, y, z], foo(x,bar(bar(y))) == bar(z)), a, b, bar(c))

# instn is substutie_vars?
# how much of this is also available from cvc5

foo(a, bar(bar(b))) == bar(bar(c))

In [53]:
help_simplify()

algebraic_number_evaluator (bool) simplify/evaluate expressions containing (algebraic) irrational numbers. (default: true)
arith_ineq_lhs (bool) rewrite inequalities so that right-hand-side is a constant. (default: false)
arith_lhs (bool) all monomials are moved to the left-hand-side, and the right-hand-side is just a constant. (default: false)
bit2bool (bool) try to convert bit-vector terms of size 1 into Boolean terms (default: true)
blast_distinct (bool) expand a distinct predicate into a quadratic number of disequalities (default: false)
blast_distinct_threshold (unsigned int) when blast_distinct is true, only distinct expressions with less than this number of arguments are blasted (default: 4294967295)
blast_eq_value (bool) blast (some) Bit-vector equalities into bits (default: false)
blast_select_store (bool) eagerly replace all (select (store ..) ..) term by an if-then-else term (default: false)
bv_extract_prop (bool) attempt to partially propagate extraction inwards (default: f

In [48]:
list(range(3,0,-1))

[3, 2, 1]

In [1]:
# It isn't all roses. Python's scoping rules are bonkers. It's intrinsically an imperative language.
match 42:
    case a:
        pass
print(a)

42


In [ ]:
def unify(a : AstRef, b : AstRef, vars):
    # or take in an existential equation.
    eqs = [(a,b)]
    subst = []
    sig = [] # miller unify?
    while eqs:
        a,b = eqs.pop()
        if a in vars:
            subst.append((a,b))
            map(lambda x: substitute(subst), eqs)
        elif b in vars:
            subst.append((b,a))
            map(lambda x: substitute(subst), eqs)
        


convert z3 ast to egglog for conditional simplification.

simp taking in z3 formula

https://microsoft.github.io/z3guide/programming/Example%20Programs/Formula%20Simplification/ examples of simplifying z3 expressions.


rules = []
def simp1(t, rules):
    for r in rules:
        match r:
            case ForAll(vars,l == r):
                subst = unify(l, r, vars)
                if subst != None:

def simp():





Term orderings


<Order.GT: 4>

ast vector and ast map are interesting.

class UnionFind():
    pass

The thing that kills me is too many design decisions. The great thing about just taking the z3 ast is it just is what it is. It is a pretty goodc design alkbeit imperfetc. In fact, it's so good it kind of inspired this whole line of thinking on my part.


https://www.philipzucker.com/programming-and-interactive-proving-with-z3py/

https://stackoverflow.com/questions/76270483/is-there-a-way-to-draw-bussproof-style-tree-diagram-in-jupyter-notebook



In [54]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Theorem():
    pass
__Theorem = Theorem
Theorem = None

@dataclass(frozen=True)
class Axiom(__Theorem):
    thm : BoolRef
#Axiom = None. # Actually. Go ahead

@dataclass(frozen=True)
class Lemma(__Theorem):
    thm : BoolRef
    by : list[Theorem]
    admit : bool
__Lemma = Lemma #This is silly? You can still get at it.
Lemma = None

def lemma(fm : BoolRef, by = [], admit = False):
    if admit:
        return __Lemma(fm, by, True)
    else:
        s = Solver()
        for n, h in enumerate(by):
            assert isinstance(h, __Theorem)
            s.add(h.thm)
            s.assert_and_track(h, f"by_{n}")
         
        s.assert_and_track(Not(fm), "goal")
        s.set("unsat_core", True)
        s.set("timeout", 1000)
        res = s.check()
        if res == unsat:
            # check the unsat core. Check it include goals
            core = s.unsat_core()
            return __Lemma(fm, by, False)
        elif res == sat:
            raise Exception("Lemma failed to prove", s.get_model())



In [ ]:
# a protocol style.
class Proof():
    thm : BoolRef
    reasons : list[Any]
    def check(self):
        pass

class Z3Lemma():
    def check(self):
        for i in self.reasons:
            i.check()
        lemma(self.thm, self.reasons)

In [ ]:
Theorem

# Proof Output Lean

There is aso that lean gym thing https://github.com/lean-dojo/LeanDojo


In [2]:
%%file /tmp/toy.lean
def main : IO Unit :=
  IO.println "Hello, world!"

Writing /tmp/toy.lean


In [4]:
! lean /tmp/toy.lean --run

Hello, world!


In [ ]:
import kdrag.smt as smt
import kdrag as kd

def expr_to_lean(t : smt.ExprRef) -> str:
    if smt.is_quantifier(t):
        vs, body = kd.utils.open_binder(t)
        vs = [f"({v.decl().name()} : {v.sort()})" for v in vs]
        if t.is_forall():  
            return f"∀ {' '.join(vs)}, {expr_to_lean(body)}"
        elif t.is_exists():
            return f"∃ {' '.join(vs)}, {expr_to_lean(body)}"
        elif t.is_lambda():
            return f"λ {' '.join(vs)} => {expr_to_lean(body)}"
    elif smt.is_eq(t):
        lhs, rhs = t.children()
        return f"{expr_to_lean(lhs)} = {expr_to_lean(rhs)}"
    elif smt.is_int_value(t):
        return str(t.as_long())
    elif smt.is_rational_value(t):    
        if t.denominator() == 1:
            return str(t.numerator())
        else:
            return f"({t.numerator()} / {t.denominator()})"
    elif smt.is_and(t):
        return f"{' ∧ '.join(expr_to_lean(c) for c in t.children())}"
    elif smt.is_or(t):
        return f"{' ∨ '.join(expr_to_lean(c) for c in t.children())}"
    elif smt.is_not(t):
        return f"¬ {expr_to_lean(t.children()[0])}"
    elif smt.is_implies(t):
        return f"{expr_to_lean(t.children()[0])} → {expr_to_lean(t.children()[1])}"
    elif smt.is_true(t):
        return "⊤"
    elif smt.is_false(t):
        return "⊥"
    elif smt.is_le(t):
        return f"{expr_to_lean(t.children()[0])} ≤ {expr_to_lean(t.children()[1])}"
    elif smt.is_lt(t):
        return f"{expr_to_lean(t.children()[0])} < {expr_to_lean(t.children()[1])}"
    elif smt.is_ge(t):
        return f"{expr_to_lean(t.children()[0])} ≥ {expr_to_lean(t.children()[1])}"
    elif smt.is_gt(t):
        return f"{expr_to_lean(t.children()[0])} > {expr_to_lean(t.children()[1])}"
    elif smt.is_add(t):
        return f"{' + '.join(expr_to_lean(c) for c in t.children())}"
    elif smt.is_sub(t):
        return f"{' - '.join(expr_to_lean(c) for c in t.children())}"
    elif smt.is_mul(t):
        return f"{' * '.join(expr_to_lean(c) for c in t.children())}"
    elif smt.is_div(t):
        return f"{' / '.join(expr_to_lean(c) for c in t.children())}"
    elif smt.is_app(t):
        decl = t.decl()
        args = [expr_to_lean(a) for a in t.children()]
        return f"{decl.name()} {' '.join(args)}"
    else:
        raise ValueError(f"Unexpected expression in expr_to_lean {t}")

    
x,y = smt.Reals("x y")
e = smt.ForAll([x,y], smt.And(x >= 7, y <= 3))
expr_to_lean(e)


def call_lean(s):
    with open("/tmp/toy.lean", "w") as f:
        f.write(s)
    return subprocess.run(["lean", "/tmp/toy.lean"], check=True)


class LeanSolver(kd.solvers.BaseSolver):
    def check(self, proof_script=None):
        # collect up all the sorts and such. 
        with open("/tmp/toy.lean", "w") as f:
            f.write(s)
            f.wrtie()
        return subprocess.run(["lean", "/tmp/toy.lean"], check=True)





'∀ (X!14 : Real) (Y!15 : Real), X!14  ≥ 7 ∧ Y!15  ≤ 3'

isabelle printer?
coq printer?
agda
haskell? (extraction?)


# Proof formats
https://www.gilith.com/opentheory/
metamath
dedukti
alethe
LF something that other cvc5

JSON using some kind of schema
hilbert style. akin to metamath
```
l # () ; comment
d     ; define
a name (a b c)
l 1 
```

proof carrying code.
I was suggesting something like this could work.

DW_ASSERT

extended smtlib format.
(define-lemma bar :by      )
(define-lemma foo :by a b c)




# Converting to TPTP


In [53]:
def collect_sig(e : ExprRef):
    sig = set()
    todo = [e]
    while todo:
        e = todo.pop()
        if is_var(e):
            continue
        elif isinstance(e, QuantifierRef):
            todo.extend(e.body())
        else:    
          sig.add(e.decl())
          todo.extend(e.children())
    return sig

E = DeclareSort("E")
x,y,z = Ints("x y z")
z,w = Consts("z w", E)
collect_sig(And(z == w, w == z))

{==, And, w, z}

;;declarations
(declare-fun f (Int) Int)
;;axioms
;;goal
(assert (not (= x x)))
(check-set)


NameError: name 'even_pf' is not defined

In [57]:
def sig_tptp_decl(decl):
    name = decl.name()
    if name in ["and", "or", "=>", "=", "ite", "not"]:
        return None
    return "thf(typedecl, type, {} : {}).".format(name, z3_sort_tptp(decl.sort()))

for decl in collect_sig(And(z == w, w == z)):
    print(decl)
    print(sig_tptp_decl(decl))


Z3Exception: sort mismatch

In [54]:




x, y = Ints("x y")
z3_to_tptp(And(x == 1, y == 2))

'((x = 1) & (y = 2))'

In [85]:
e= Not(BoolVal(False))
e.sort_kind()
e.decl().kind() == z3.Z3_OP_NOT
f(x).decl().kind() == z3.Z3_OP_UNINTERPRETED
BoolVal(False).decl().kind() == z3.Z3_OP_FALSE

True

In [ ]:
_z3_op_to_tptp = {
    Z3_OP_TRUE: "$true",
    Z3_OP_FALSE: "$false",
    Z3_OP_EQ: "=",
    Z3_OP_DISTINCT: "Distinct",
    Z3_OP_ITE: "$ite",
    Z3_OP_AND: "&",
    Z3_OP_OR: "|",
    Z3_OP_IFF: "<=>",
    Z3_OP_XOR: "Xor",
    Z3_OP_NOT: "~",
    Z3_OP_IMPLIES: "=>",

    Z3_OP_IDIV: "/",
    Z3_OP_MOD: "%",
    Z3_OP_TO_REAL: "ToReal",
    Z3_OP_TO_INT: "ToInt",
    Z3_OP_POWER: "**",
    Z3_OP_IS_INT: "IsInt",
    Z3_OP_BADD: "+",
    Z3_OP_BSUB: "-",
    Z3_OP_BMUL: "*",
    Z3_OP_BOR: "|",
    Z3_OP_BAND: "&",
    Z3_OP_BNOT: "~",
    Z3_OP_BXOR: "^",
    Z3_OP_BNEG: "-",
    Z3_OP_BUDIV: "UDiv",
    Z3_OP_BSDIV: "/",
    Z3_OP_BSMOD: "%",
    Z3_OP_BSREM: "SRem",
    Z3_OP_BUREM: "URem",

    Z3_OP_EXT_ROTATE_LEFT: "RotateLeft",
    Z3_OP_EXT_ROTATE_RIGHT: "RotateRight",

    Z3_OP_SLEQ: "<=",
    Z3_OP_SLT: "<",
    Z3_OP_SGEQ: ">=",
    Z3_OP_SGT: ">",

    Z3_OP_ULEQ: "ULE",
    Z3_OP_ULT: "ULT",
    Z3_OP_UGEQ: "UGE",
    Z3_OP_UGT: "UGT",
    Z3_OP_SIGN_EXT: "SignExt",
    Z3_OP_ZERO_EXT: "ZeroExt",

    Z3_OP_REPEAT: "RepeatBitVec",
    Z3_OP_BASHR: ">>",
    Z3_OP_BSHL: "<<",
    Z3_OP_BLSHR: "LShR",

    Z3_OP_CONCAT: "Concat",
    Z3_OP_EXTRACT: "Extract",
    Z3_OP_BV2INT: "BV2Int",
    Z3_OP_ARRAY_MAP: "Map",
    Z3_OP_SELECT: "Select",
    Z3_OP_STORE: "Store",
    Z3_OP_CONST_ARRAY: "K",
    Z3_OP_ARRAY_EXT: "Ext",

    Z3_OP_PB_AT_MOST: "AtMost",
    Z3_OP_PB_AT_LEAST: "AtLeast",
    Z3_OP_PB_LE: "PbLe",
    Z3_OP_PB_GE: "PbGe",
    Z3_OP_PB_EQ: "PbEq",

    Z3_OP_SEQ_CONCAT: "Concat",
    Z3_OP_SEQ_PREFIX: "PrefixOf",
    Z3_OP_SEQ_SUFFIX: "SuffixOf",
    Z3_OP_SEQ_UNIT: "Unit",
    Z3_OP_SEQ_CONTAINS: "Contains",
    Z3_OP_SEQ_REPLACE: "Replace",
    Z3_OP_SEQ_AT: "At",
    Z3_OP_SEQ_NTH: "Nth",
    Z3_OP_SEQ_INDEX: "IndexOf",
    Z3_OP_SEQ_LAST_INDEX: "LastIndexOf",
    Z3_OP_SEQ_LENGTH: "Length",
    Z3_OP_STR_TO_INT: "StrToInt",
    Z3_OP_INT_TO_STR: "IntToStr",

    Z3_OP_SEQ_IN_RE: "InRe",
    Z3_OP_SEQ_TO_RE: "Re",
    Z3_OP_RE_PLUS: "Plus",
    Z3_OP_RE_STAR: "Star",
    Z3_OP_RE_OPTION: "Option",
    Z3_OP_RE_UNION: "Union",
    Z3_OP_RE_RANGE: "Range",
    Z3_OP_RE_INTERSECT: "Intersect",
    Z3_OP_RE_COMPLEMENT: "Complement",

    Z3_OP_FPA_IS_NAN: "fpIsNaN",
    Z3_OP_FPA_IS_INF: "fpIsInf",
    Z3_OP_FPA_IS_ZERO: "fpIsZero",
    Z3_OP_FPA_IS_NORMAL: "fpIsNormal",
    Z3_OP_FPA_IS_SUBNORMAL: "fpIsSubnormal",
    Z3_OP_FPA_IS_NEGATIVE: "fpIsNegative",
    Z3_OP_FPA_IS_POSITIVE: "fpIsPositive",
}

In [55]:
def z3_problem_to_tptp(probs : list[BoolRef]):
    tptp = []
    sig = {d for prob in probs for d in collect_sig(prob)}
    tptp.extend(map(sig_tptp_decl, sig))
    tptp.extend(map(z3_to_tptp, probs))
    return tptp

x, y, z = Ints("x y z")
z3_problem_to_tptp([x == y, y == z])

    


AttributeError: 'FuncDeclRef' object has no attribute 'sort'

In [112]:
Exists([x], x == x).is_exists()
substitute_vars(Lambda([x,y], x + y).body(), IntVal(3))
Select(Lambda([x], x + 1), 10)

AttributeError: 'QuantifierRef' object has no attribute 'domain_n'

In [69]:
x.decl().name()

'x'

In [73]:
# nice smt printing. Using solver kind of mangles it
from z3 import *
x = Int("x")
f = Function("f", IntSort(), IntSort())
x.decl().sexpr()

def sexpr_datatype(dt):
    print(f"(declare-datatypes ({dt.name()}))")
    for c in dt.constructor():
        print(f"({c.name()} {c.sort()})")
    #print(f"(declare-fun is_{dt.name()} (E) Bool)")
    #print(f"(assert (forall ((x E)) (= (is_{dt.name()} x) (or {c.name()} x))))")


import subprocess

def lemma_smt(thm, by=[], sig=[]):
    output = []
    output.append(";;declarations")
    for f in sig:
        output.append(f.sexpr())
    output.append(";;axioms")
    for e in by:
        output.append("(assert " + e.sexpr() + ")")
    output.append(";;goal")
    output.append("(assert " + Not(thm).sexpr() + ")")
    output.append("(check-set)")
    return output

def lemma_ext(thm, by=[], sig=[], command=None, timeout=1000):
    # TODO use better temporary file
    with open("/tmp/temp.smt2", "w") as f:
        smt = lemma_smt(thm, by=by, sig=sig)
        f.writelines(smt)
    command.append("/tmp/temp.smt2")
    res = subprocess.run(
        command,
        timeout=1.0,
        capture_output=True,
    )
    return res

def lemma_vampire(thm, by=[], sig=[], timeout=1000):
    return lemma_ext(thm, by=by, sig=sig, command=["vampire", "--output_mode", "smtcomp"], timeout=timeout)

lemma_vampire(f(7) == 7, sig=[f]).stdout

;;declarations
(declare-fun f (Int) Int)
;;axioms
;;goal
(assert (not (= x x)))
(check-set)


b'Running in auto input_syntax mode. Trying SMTLIB2\nsat\n'